## CNC Mill Wear Prediction Using Reinforcement Learning and Data from PHM 2010  

In [ ]:
pip install torch torchvision torchaudio

In [ ]:
import torch

if torch.cuda.is_available():
    print("CUDA is available! Using GPU.")
else:
    print("CUDA is not available. Using CPU.")

In [ ]:
pip install stable-baselines3[extra]

In [ ]:
pip install gymnasium

In [ ]:
# Helper Libraries
import matplotlib.pyplot as plt
import numpy as np
import random
import pandas as pd
import os
import torch
import time

# Environment Building
import gymnasium
from gymnasium import Env
from gymnasium.spaces import Discrete, Box, Dict, Tuple, MultiBinary, MultiDiscrete

# RL Algorithms
import stable_baselines3
from stable_baselines3 import PPO
from stable_baselines3 import A2C
from stable_baselines3 import DDPG
from stable_baselines3 import SAC

from stable_baselines3.common.env_checker import check_env
from stable_baselines3.common.vec_env import VecFrameStack
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.results_plotter import plot_results, X_TIMESTEPS
from stable_baselines3.common.monitor import Monitor

from pathlib import Path

In [ ]:
file_paths_cnc = "./normalised_time_domain_features_with_cut_and_flute_max.csv"
df = pd.read_csv(file_paths_cnc)
print(df.head())

In [ ]:
# Global Parameters for CNC Custom Env

MAX_STEPS = len(df.loc[:, 'File'].values) #len = 315

MAX_DIFF = 6

REW_WEIGHT = [0.1 , 0.1 , 0.1 , 0.7] # flute_1-2-3 and flute_max

time_domain_features_columns = ['dyn_X_mean', 'dyn_X_rms', 'dyn_X_crest', 'dyn_X_avg_power', 'dyn_X_skewness', 'dyn_X_kurtosis', 'dyn_Y_mean', 'dyn_Y_rms', 'dyn_Y_crest', 'dyn_Y_avg_power', 'dyn_Y_skewness', 'dyn_Y_kurtosis', 'dyn_Z_mean', 'dyn_Z_rms', 'dyn_Z_crest', 'dyn_Z_avg_power', 'dyn_Z_skewness', 'dyn_Z_kurtosis', 'acc_X_mean', 'acc_X_rms', 'acc_X_crest', 'acc_X_avg_power', 'acc_X_skewness', 'acc_X_kurtosis', 'acc_Y_mean', 'acc_Y_rms', 'acc_Y_crest', 'acc_Y_avg_power', 'acc_Y_skewness', 'acc_Y_kurtosis', 'acc_Z_mean', 'acc_Z_rms', 'acc_Z_crest', 'acc_Z_avg_power', 'acc_Z_skewness', 'acc_Z_kurtosis', 'AE_mean', 'AE_rms', 'AE_crest', 'AE_avg_power', 'AE_skewness', 'AE_kurtosis']
flute_wear_collumns = ['flute_1','flute_2', 'flute_3', 'flute_max']

In [ ]:
''''class CNC_Env_Non_Corrective_Pred(gymnasium.Env):

    def __init__(self, df):

        self.df = df
        self.reward_range = (np.NINF , 1 ) # Πρέπει να ορίσω το εύρος των rewards

        # Define the low and high bounds for each dimension of the action space
        low_bound = np.array([0, 0, 0], dtype=np.float64)
        high_bound = np.array([1, 1, 1], dtype=np.float64)

        # Action Space
        self.action_space = gymnasium.spaces.Box(low=low_bound, high=high_bound, dtype=np.float64)

        # Observation Space | Είναι κανονικοποιημένο από 0-1 | 7 sensors * 6 time domain features = 42
        self.observation_space = gymnasium.spaces.Box(low=0, high=1, shape=(42,), dtype=np.float64)

        # Ξεκινάω το περιβάλλον στην αρχική του κατάσταση (state s0)
        self._state_s0()

    def _state_s0(self):
        # βάλε randomness εδώ
        self.current_step = 0
        # flute_1 = df[] diff bettwen d[1] - d[0] * (random val (0,2))
        # state_0 - ( state_1 - state_0)
        self.flute_1 = (2*self.df.loc[0, "flute_1"]) - self.df.loc[1, "flute_1"]
        self.flute_2 = (2*self.df.loc[0, "flute_2"]) - self.df.loc[1, "flute_2"]
        self.flute_3 =  (2*self.df.loc[0, "flute_3"]) - self.df.loc[1, "flute_3"]
        self.flute_max = (2*self.df.loc[0, "flute_max"]) - self.df.loc[1, "flute_max"]

        # Θέτουμε και τα predictions στην αρχική κατάσταση του συστήματος
        self.flute_1_pred = self.flute_1
        self.flute_2_pred = self.flute_2
        self.flute_3_pred = self.flute_3
        self.flute_max_pred = max(self.flute_1 , self.flute_2 , self.flute_3)

        self.pred_dif_1 = 0
        self.pred_dif_2 = 0
        self.pred_dif_3 = 0
        self.pred_dif_max = 0

    # Constructing Observations From Environment States
    def _get_obs(self):

        # Παίρνω τα time domain features του current (δηλαδή του αντίστοιχου cut)
        frame = self.df.loc[self.current_step, time_domain_features_columns].to_numpy(dtype=np.float64)

        # Φτιαχνω το observation
        obs = frame

        return obs

    def _get_info(self):
        return {
            "cut": self.current_step
        }

    def _take_action(self, action):

        # Βρες την διαφορά μεταξύ της προηγούμενης τιμής και της εκτιμόμενης τιμής

        # Action Space Example : array([0.79758393, 0.41074004, 0.0026002 ])

        # Αν θέλουμε μετά από κάθε state Sn να κάνουμε ένα Predict βάση του σωστού Sn-1
        #self.flute_1_pred = self.flute_1 + (action[0] * MAX_DIFF)
        #self.flute_2_pred = self.flute_2 + (action[1] * MAX_DIFF)
        #self.flute_3_pred = self.flute_3 + (action[2] * MAX_DIFF)
        #self.flute_max_pred = max(self.flute_1_pred , self.flute_2_pred , self.flute_3_pred)

        ## Σε κάθε state Sn κάνουμε Predict βάση της προηγούμενης πρόβλεψης Sn-1
        #print("Before : ", self.flute_1_pred)
        # φθορα στο μυτακι 1 (ν) = φθορα στο μυτακι 1 πρβλεπομενη(ν-1) + prediction του agent
        # Prediction του agent = 100% max diff φθορων σε καταστασεις χρησεις του μηχανηματος (ν -> ν+1)
        self.flute_1_pred += (action[0] * MAX_DIFF)
        #print("After : ", self.flute_1_pred)
        self.flute_2_pred += (action[1] * MAX_DIFF)
        self.flute_3_pred += (action[2] * MAX_DIFF)
        self.flute_max_pred = max(self.flute_1_pred , self.flute_2_pred , self.flute_3_pred)

        self.flute_1 = self.df.loc[self.current_step, "flute_1"]
        self.flute_2 = self.df.loc[self.current_step, "flute_2"]
        self.flute_3 = self.df.loc[self.current_step, "flute_3"]
        self.flute_max = self.df.loc[self.current_step, "flute_max"]

        # Η διαφορά μεταξύ της προβλεπόμενης τιμής και της κανονικής τιμής
        self.pred_dif_1 = self.flute_1_pred - self.flute_1
        self.pred_dif_2 = self.flute_1_pred - self.flute_1
        self.pred_dif_3 = self.flute_1_pred - self.flute_1
        self.pred_dif_max = self.flute_max_pred - self.flute_max

    def reset(self, seed = None, options = None):
        #We need the following line to seed self.np_random
        super().reset(seed=seed)

        # Επιστρέφω το περιβάλλον στην αρχική του κατάσταση (state s0)
        self._state_s0()

        # Set the current step to the start of the dataset
        self.current_step = 0

        observation = self._get_obs()
        info = self._get_info()

        return observation,info

    def score_function(self,delta):
        if delta < 0:
            return 1 - np.exp(-delta/10)
        else:
            return  1 - np.exp(delta/4.5)

    def step(self, action):
        # Execute one time step within the environment
        self._take_action(action)

        self.current_step += 1

        # Παραμετροποιεί το reward έτσι ώστε να ενισχύει τις δράσεις που προσφέρουν οφέλοι σε μέλλοντα χρόνο
        #delay_modifier = (self.current_step / MAX_STEPS)

        reward_flute_1 = self.score_function(self.pred_dif_1)
        reward_flute_2 = self.score_function(self.pred_dif_2)
        reward_flute_3 = self.score_function(self.pred_dif_3)
        reward_flute_max = self.score_function(self.pred_dif_max)

        reward = (REW_WEIGHT[0]*reward_flute_1) + (REW_WEIGHT[1]*reward_flute_2) + (REW_WEIGHT[2]*reward_flute_3) + (REW_WEIGHT[3]*reward_flute_max)
        #reward = reward * delay_modifier
        #print("Reward : " ,reward)
        # Ελέγχουμε εάν πάμε να βγούμε εκτός του dataset
        terminated = False
        if self.current_step > MAX_STEPS - 2:
            # τότε πρέπει να κάνουμε reset
            terminated = True  #διαλέγω terminated καθώς είναι φυσιολογική η λήξη

        observation = self._get_obs()
        info = self._get_info()

        #Εδω μπορούμε να βάλουμε κάποιο όριο απόκλισης τιμών στις προβλέψεις ώστε να κάνουμε την μάθηση ταχύτερη
        trunctated = False # Ελέγχουμε εάν υπάρχει κάποιο condition εξωτερικό του περιβάλλοντος πχ. Όριο χρόνου
        if reward < -5000 :
          trunctated = True

        return observation, reward, terminated,trunctated, info

    metadata = {"render_modes": ["human"]} #, "rgb_array"], "render_fps": 4}

    def render(self, mode='human'):
        # Render the environment (optional)
        pass

    def close(self):
        # Clean up resources (optional)
        pass

In [ ]:
''''class CNC_Env_Corrective_Pred(gymnasium.Env):

    def __init__(self, df):

        self.df = df
        self.reward_range = (np.NINF , 1 ) # Πρέπει να ορίσω το εύρος των rewards

        # Define the low and high bounds for each dimension of the action space
        low_bound = np.array([0, 0, 0], dtype=np.float64)
        high_bound = np.array([1, 1, 1], dtype=np.float64)

        # Action Space
        self.action_space = gymnasium.spaces.Box(low=low_bound, high=high_bound, dtype=np.float64)

        # Observation Space | Είναι κανονικοποιημένο από 0-1
        self.observation_space = gymnasium.spaces.Box(low=0, high=1, shape=(42,), dtype=np.float64)

        # Ξεκινάω το περιβάλλον στην αρχική του κατάσταση (state s0)
        self._state_s0()

    def _state_s0(self):
        # βάλε randomness εδώ
        self.current_step = 0
        # flute_1 = df[] diff bettwen d[1] - d[0] * (random val (0,2))
        # state_0 - ( state_1 - state_0)
        self.flute_1 = (2*self.df.loc[0, "flute_1"]) - self.df.loc[1, "flute_1"]
        self.flute_2 = (2*self.df.loc[0, "flute_2"]) - self.df.loc[1, "flute_2"]
        self.flute_3 =  (2*self.df.loc[0, "flute_3"]) - self.df.loc[1, "flute_3"]
        self.flute_max = (2*self.df.loc[0, "flute_max"]) - self.df.loc[1, "flute_max"]

        # Θέτουμε και τα predictions στην αρχική κατάσταση του συστήματος
        self.flute_1_pred = self.flute_1
        self.flute_2_pred = self.flute_2
        self.flute_3_pred = self.flute_3
        self.flute_max_pred = max(self.flute_1 , self.flute_2 , self.flute_3)

        self.pred_dif_1 = 0
        self.pred_dif_2 = 0
        self.pred_dif_3 = 0
        self.pred_dif_max = 0

    # Constructing Observations From Environment States
    def _get_obs(self):

        # Παίρνω τα time domain features του current (δηλαδή του αντίστοιχου cut)
        frame = self.df.loc[self.current_step, time_domain_features_columns].to_numpy(dtype=np.float64)

        # Φτιαχνω το observation
        obs = frame

        return obs

    def _get_info(self):
        return {
            "cut": self.current_step
        }

    def _take_action(self, action):

        # Βρες την διαφορά μεταξύ της προηγούμενης τιμής και της εκτιμόμενης τιμής

        # Action Space Example : array([0.79758393, 0.41074004, 0.0026002 ])

        # Αν θέλουμε μετά από κάθε state Sn να κάνουμε ένα Predict βάση του σωστού Sn-1
        self.flute_1_pred = self.flute_1 + (action[0] * MAX_DIFF)
        self.flute_2_pred = self.flute_2 + (action[1] * MAX_DIFF)
        self.flute_3_pred = self.flute_3 + (action[2] * MAX_DIFF)
        self.flute_max_pred = max(self.flute_1_pred , self.flute_2_pred , self.flute_3_pred)

        ## Σε κάθε state Sn κάνουμε Predict βάση της προηγούμενης πρόβλεψης Sn-1
        #self.flute_1_pred = self.flute_1_pred + (action[0] * MAX_DIFF)    # !!!!!!!! flute_1_pred
        #self.flute_2_pred = self.flute_2_pred + (action[1] * MAX_DIFF)
        #self.flute_3_pred = self.flute_3_pred + (action[2] * MAX_DIFF)
        #self.flute_max_pred = max(self.flute_1_pred , self.flute_2_pred , self.flute_3_pred)

        self.flute_1 = self.df.loc[self.current_step, "flute_1"]
        self.flute_2 = self.df.loc[self.current_step, "flute_2"]
        self.flute_3 = self.df.loc[self.current_step, "flute_3"]
        self.flute_max = self.df.loc[self.current_step, "flute_max"]

        # Η διαφορά μεταξύ της προβλεπόμενης τιμής και της κανονικής τιμής
        self.pred_dif_1 = self.flute_1_pred - self.flute_1
        self.pred_dif_2 = self.flute_1_pred - self.flute_1
        self.pred_dif_3 = self.flute_1_pred - self.flute_1
        self.pred_dif_max = self.flute_max_pred - self.flute_max

    def reset(self, seed = None, options = None):
        #We need the following line to seed self.np_random
        super().reset(seed=seed)

        # Επιστρέφω το περιβάλλον στην αρχική του κατάσταση (state s0)
        self._state_s0()

        # Set the current step to the start of the dataset
        self.current_step = 0

        observation = self._get_obs()
        info = self._get_info()

        return observation,info

    def score_function(self,delta):
        if delta < 0:
            return 1 - np.exp(-delta/10)
        else:
            return  1 - np.exp(delta/4.5)

    def step(self, action):
        # Execute one time step within the environment
        self._take_action(action)

        self.current_step += 1

        # Παραμετροποιεί το reward έτσι ώστε να ενισχύει τις δράσεις που προσφέρουν οφέλοι σε μέλλοντα χρόνο
        #delay_modifier = (self.current_step / MAX_STEPS)

        reward_flute_1 = self.score_function(self.pred_dif_1)
        reward_flute_2 = self.score_function(self.pred_dif_2)
        reward_flute_3 = self.score_function(self.pred_dif_3)
        reward_flute_max = self.score_function(self.pred_dif_max)

        reward = (REW_WEIGHT[0]*reward_flute_1) + (REW_WEIGHT[1]*reward_flute_2) + (REW_WEIGHT[2]*reward_flute_3) + (REW_WEIGHT[3]*reward_flute_max)
        #reward = reward * delay_modifier

        # Ελέγχουμε εάν πάμε να βγούμε εκτός του dataset
        terminated = False
        if self.current_step > MAX_STEPS - 2:
            # τότε πρέπει να κάνουμε reset
            terminated = True  #διαλέγω terminated καθώς είναι φυσιολογική η λήξη

        observation = self._get_obs()
        info = self._get_info()

        #Εδω μπορούμε να βάλουμε κάποιο όριο απόκλισης τιμών στις προβλέψεις ώστε να κάνουμε την μάθηση ταχύτερη
        trunctated = False # Ελέγχουμε εάν υπάρχει κάποιο condition εξωτερικό του περιβάλλοντος πχ. Όριο χρόνου

        return observation, reward, terminated,trunctated, info

    metadata = {"render_modes": ["human"]} #, "rgb_array"], "render_fps": 4}

    def render(self, mode='human'):
        # Render the environment (optional)
        pass

    def close(self):
        # Clean up resources (optional)
        pass

In [ ]:
class CNC_Env_Corrective_Pred_No_Delay(gymnasium.Env):

    def __init__(self, df):

        self.df = df
        self.reward_range = (np.NINF , 1 ) # Πρέπει να ορίσω το εύρος των rewards

        # Define the low and high bounds for each dimension of the action space
        low_bound = np.array([0, 0, 0], dtype=np.float64)
        high_bound = np.array([1, 1, 1], dtype=np.float64)

        # Action Space
        self.action_space = gymnasium.spaces.Box(low=low_bound, high=high_bound, dtype=np.float64)

        # Observation Space | Είναι κανονικοποιημένο από 0-1
        self.observation_space = gymnasium.spaces.Box(low=0, high=1, shape=(42,), dtype=np.float64)

        # Ξεκινάω το περιβάλλον στην αρχική του κατάσταση (state s0)
        self._state_s0()

    def _state_s0(self):
        # βάλε randomness εδώ
        self.current_step = 0
        # flute_1 = df[] diff bettwen d[1] - d[0] * (random val (0,2))
        # state_0 - ( state_1 - state_0)
        self.flute_1 = (2*self.df.loc[0, "flute_1"]) - self.df.loc[1, "flute_1"]
        self.flute_2 = (2*self.df.loc[0, "flute_2"]) - self.df.loc[1, "flute_2"]
        self.flute_3 =  (2*self.df.loc[0, "flute_3"]) - self.df.loc[1, "flute_3"]
        self.flute_max = (2*self.df.loc[0, "flute_max"]) - self.df.loc[1, "flute_max"]

        # Θέτουμε και τα predictions στην αρχική κατάσταση του συστήματος
        self.flute_1_pred = self.flute_1
        self.flute_2_pred = self.flute_2
        self.flute_3_pred = self.flute_3
        self.flute_max_pred = max(self.flute_1 , self.flute_2 , self.flute_3)

        self.pred_dif_1 = 0
        self.pred_dif_2 = 0
        self.pred_dif_3 = 0
        self.pred_dif_max = 0
        
        #Phm score
        self.score = 0 
        

    # Constructing Observations From Environment States
    def _get_obs(self):

        # Παίρνω τα time domain features του current (δηλαδή του αντίστοιχου cut)
        frame = self.df.loc[self.current_step, time_domain_features_columns].to_numpy(dtype=np.float64)

        # Φτιαχνω το observation
        obs = frame

        return obs

    def _get_info(self):
        return {
            "cut": self.current_step,
            "score": self.score,
        }
    

    def _take_action(self, action):

        # Βρες την διαφορά μεταξύ της προηγούμενης τιμής και της εκτιμόμενης τιμής

        # Action Space Example : array([0.79758393, 0.41074004, 0.0026002 ])

        # Αν θέλουμε μετά από κάθε state Sn να κάνουμε ένα Predict βάση του σωστού Sn-1
        self.flute_1_pred = self.flute_1 + (action[0] * MAX_DIFF)
        self.flute_2_pred = self.flute_2 + (action[1] * MAX_DIFF)
        self.flute_3_pred = self.flute_3 + (action[2] * MAX_DIFF)
        self.flute_max_pred = max(self.flute_1_pred , self.flute_2_pred , self.flute_3_pred)

        self.flute_1 = self.df.loc[self.current_step, "flute_1"]
        self.flute_2 = self.df.loc[self.current_step, "flute_2"]
        self.flute_3 = self.df.loc[self.current_step, "flute_3"]
        self.flute_max = self.df.loc[self.current_step, "flute_max"]

        # Η διαφορά μεταξύ της προβλεπόμενης τιμής και της κανονικής τιμής
        self.pred_dif_1 = self.flute_1_pred - self.flute_1
        self.pred_dif_2 = self.flute_1_pred - self.flute_1
        self.pred_dif_3 = self.flute_1_pred - self.flute_1
        self.pred_dif_max = self.flute_max_pred - self.flute_max
        
        
    def reset(self, seed = None, options = None):
        #We need the following line to seed self.np_random
        super().reset(seed=seed)

        # Επιστρέφω το περιβάλλον στην αρχική του κατάσταση (state s0)
        self._state_s0()

        # Set the current step to the start of the dataset
        self.current_step = 0

        observation = self._get_obs()
        info = self._get_info()

        return observation,info
    
    def score_function_original(self, delta):
        if delta < 0:
            return np.exp(-delta/10) - 1
        else:
            return np.exp(delta/4.5) - 1

    def score_function(self,delta):
        if delta < 0:
            return 1 - np.exp(-delta/10)
        else:
            return  1 - np.exp(delta/4.5)

    def step(self, action):
        # Execute one time step within the environment
        self._take_action(action)

        self.current_step += 1

        # Παραμετροποιεί το reward έτσι ώστε να ενισχύει τις δράσεις που προσφέρουν οφέλοι σε μέλλοντα χρόνο
        #delay_modifier = (self.current_step / MAX_STEPS)

        reward_flute_1 = self.score_function(self.pred_dif_1)
        reward_flute_2 = self.score_function(self.pred_dif_2)
        reward_flute_3 = self.score_function(self.pred_dif_3)
        reward_flute_max = self.score_function(self.pred_dif_max)

        reward = (REW_WEIGHT[0]*reward_flute_1) + (REW_WEIGHT[1]*reward_flute_2) + (REW_WEIGHT[2]*reward_flute_3) + (REW_WEIGHT[3]*reward_flute_max)
        #reward = reward * delay_modifier
        
        # H βαθμολογία που θα είχε στο διαγωνισμό
        self.score += self.score_function_original(self.pred_dif_max)
        
        info = self._get_info()
        
        # Ελέγχουμε εάν πάμε να βγούμε εκτός του dataset
        terminated = False
        if self.current_step > MAX_STEPS - 2:
            # τότε πρέπει να κάνουμε reset
            #print("Terminated at :" self.current_step)
            terminated = True  #διαλέγω terminated καθώς είναι φυσιολογική η λήξη

        observation = self._get_obs()

        #Εδω μπορούμε να βάλουμε κάποιο όριο απόκλισης τιμών στις προβλέψεις ώστε να κάνουμε την μάθηση ταχύτερη
        trunctated = False # Ελέγχουμε εάν υπάρχει κάποιο condition εξωτερικό του περιβάλλοντος πχ. Όριο χρόνου

        return observation, reward, terminated,trunctated, info

    metadata = {"render_modes": ["human"]} #, "rgb_array"], "render_fps": 4}

    def render(self, mode='human'):
        # Render the environment (optional)
        pass

    def close(self):
        # Clean up resources (optional)
        pass

In [ ]:
class CNC_Env_Corrective_Pred_with_Delay(gymnasium.Env):

    def __init__(self, df):

        self.df = df
        self.reward_range = (np.NINF , 1 ) # Πρέπει να ορίσω το εύρος των rewards

        # Define the low and high bounds for each dimension of the action space
        low_bound = np.array([0, 0, 0], dtype=np.float64)
        high_bound = np.array([1, 1, 1], dtype=np.float64)

        # Action Space
        self.action_space = gymnasium.spaces.Box(low=low_bound, high=high_bound, dtype=np.float64)

        # Observation Space | Είναι κανονικοποιημένο από 0-1
        self.observation_space = gymnasium.spaces.Box(low=0, high=1, shape=(42,), dtype=np.float64)

        # Ξεκινάω το περιβάλλον στην αρχική του κατάσταση (state s0)
        self._state_s0()

    def _state_s0(self):
        # βάλε randomness εδώ
        self.current_step = 0
        # flute_1 = df[] diff bettwen d[1] - d[0] * (random val (0,2))
        # state_0 - ( state_1 - state_0)
        self.flute_1 = (2*self.df.loc[0, "flute_1"]) - self.df.loc[1, "flute_1"]
        self.flute_2 = (2*self.df.loc[0, "flute_2"]) - self.df.loc[1, "flute_2"]
        self.flute_3 =  (2*self.df.loc[0, "flute_3"]) - self.df.loc[1, "flute_3"]
        self.flute_max = (2*self.df.loc[0, "flute_max"]) - self.df.loc[1, "flute_max"]

        # Θέτουμε και τα predictions στην αρχική κατάσταση του συστήματος
        self.flute_1_pred = self.flute_1
        self.flute_2_pred = self.flute_2
        self.flute_3_pred = self.flute_3
        self.flute_max_pred = max(self.flute_1 , self.flute_2 , self.flute_3)

        self.pred_dif_1 = 0
        self.pred_dif_2 = 0
        self.pred_dif_3 = 0
        self.pred_dif_max = 0
        
        #Phm score
        self.score = 0 

    # Constructing Observations From Environment States
    def _get_obs(self):

        # Παίρνω τα time domain features του current (δηλαδή του αντίστοιχου cut)
        frame = self.df.loc[self.current_step, time_domain_features_columns].to_numpy(dtype=np.float64)

        # Φτιαχνω το observation
        obs = frame

        return obs

    def _get_info(self):
        return {
            "cut": self.current_step,
            "score": self.score
        }
    

    def _take_action(self, action):

        # Βρες την διαφορά μεταξύ της προηγούμενης τιμής και της εκτιμόμενης τιμής

        # Action Space Example : array([0.79758393, 0.41074004, 0.0026002 ])

        # Αν θέλουμε μετά από κάθε state Sn να κάνουμε ένα Predict βάση του σωστού Sn-1
        self.flute_1_pred = self.flute_1 + (action[0] * MAX_DIFF)
        self.flute_2_pred = self.flute_2 + (action[1] * MAX_DIFF)
        self.flute_3_pred = self.flute_3 + (action[2] * MAX_DIFF)
        self.flute_max_pred = max(self.flute_1_pred , self.flute_2_pred , self.flute_3_pred)

        self.flute_1 = self.df.loc[self.current_step, "flute_1"]
        self.flute_2 = self.df.loc[self.current_step, "flute_2"]
        self.flute_3 = self.df.loc[self.current_step, "flute_3"]
        self.flute_max = self.df.loc[self.current_step, "flute_max"]

        # Η διαφορά μεταξύ της προβλεπόμενης τιμής και της κανονικής τιμής
        self.pred_dif_1 = self.flute_1_pred - self.flute_1
        self.pred_dif_2 = self.flute_1_pred - self.flute_1
        self.pred_dif_3 = self.flute_1_pred - self.flute_1
        self.pred_dif_max = self.flute_max_pred - self.flute_max
        
        
    def reset(self, seed = None, options = None):
        #We need the following line to seed self.np_random
        super().reset(seed=seed)

        # Επιστρέφω το περιβάλλον στην αρχική του κατάσταση (state s0)
        self._state_s0()

        # Set the current step to the start of the dataset
        self.current_step = 0

        observation = self._get_obs()
        info = self._get_info()

        return observation,info
    
    def score_function_original(self, delta):
        if delta < 0:
            return np.exp(-delta/10) - 1
        else:
            return np.exp(delta/4.5) - 1

    def score_function(self,delta):
        if delta < 0:
            return 1 - np.exp(-delta/10)
        else:
            return  1 - np.exp(delta/4.5)

    def step(self, action):
        # Execute one time step within the environment
        self._take_action(action)

        self.current_step += 1

        reward_flute_1 = self.score_function(self.pred_dif_1)
        reward_flute_2 = self.score_function(self.pred_dif_2)
        reward_flute_3 = self.score_function(self.pred_dif_3)
        reward_flute_max = self.score_function(self.pred_dif_max)

        reward_without_delay = (REW_WEIGHT[0]*reward_flute_1) + (REW_WEIGHT[1]*reward_flute_2) + (REW_WEIGHT[2]*reward_flute_3) + (REW_WEIGHT[3]*reward_flute_max)
        # Παραμετροποιεί το reward έτσι ώστε να ενισχύει τις δράσεις που προσφέρουν οφέλοι σε μέλλοντα χρόνο
        delay_modifier = (self.current_step / (MAX_STEPS-2))
        reward = reward_without_delay * delay_modifier
        
        # H βαθμολογία που θα είχε στο διαγωνισμό
        self.score += self.score_function_original(self.pred_dif_max)
        
        info = self._get_info()
        
        # Ελέγχουμε εάν πάμε να βγούμε εκτός του dataset
        terminated = False
        if self.current_step > MAX_STEPS - 2:
            # τότε πρέπει να κάνουμε reset
            terminated = True  #διαλέγω terminated καθώς είναι φυσιολογική η λήξη

        observation = self._get_obs()

        #Εδω μπορούμε να βάλουμε κάποιο όριο απόκλισης τιμών στις προβλέψεις ώστε να κάνουμε την μάθηση ταχύτερη
        trunctated = False # Ελέγχουμε εάν υπάρχει κάποιο condition εξωτερικό του περιβάλλοντος πχ. Όριο χρόνου

        return observation, reward, terminated,trunctated, info

    metadata = {"render_modes": ["human"]} #, "rgb_array"], "render_fps": 4}

    def render(self, mode='human'):
        # Render the environment (optional)
        pass

    def close(self):
        # Clean up resources (optional)
        pass

In [ ]:
class CNC_Env_Non_Corrective_No_Delay(gymnasium.Env):

    def __init__(self, df):

        self.df = df
        self.reward_range = (np.NINF , 1 ) # Πρέπει να ορίσω το εύρος των rewards

        # Define the low and high bounds for each dimension of the action space
        low_bound = np.array([0, 0, 0], dtype=np.float64)
        high_bound = np.array([1, 1, 1], dtype=np.float64)

        # Action Space
        self.action_space = gymnasium.spaces.Box(low=low_bound, high=high_bound, dtype=np.float64)

        # Observation Space | Είναι κανονικοποιημένο από 0-1 | 7 sensors * 6 time domain features = 42
        self.observation_space = gymnasium.spaces.Box(low=0, high=1, shape=(42,), dtype=np.float64)

        # Ξεκινάω το περιβάλλον στην αρχική του κατάσταση (state s0)
        self._state_s0()

    def _state_s0(self):
        # βάλε randomness εδώ
        self.current_step = 0
        # flute_1 = df[] diff bettwen d[1] - d[0] * (random val (0,2))
        # state_0 - ( state_1 - state_0)
        self.flute_1 = (2*self.df.loc[0, "flute_1"]) - self.df.loc[1, "flute_1"]
        self.flute_2 = (2*self.df.loc[0, "flute_2"]) - self.df.loc[1, "flute_2"]
        self.flute_3 =  (2*self.df.loc[0, "flute_3"]) - self.df.loc[1, "flute_3"]
        self.flute_max = (2*self.df.loc[0, "flute_max"]) - self.df.loc[1, "flute_max"]

        # Θέτουμε και τα predictions στην αρχική κατάσταση του συστήματος
        self.flute_1_pred = self.flute_1
        self.flute_2_pred = self.flute_2
        self.flute_3_pred = self.flute_3
        self.flute_max_pred = max(self.flute_1 , self.flute_2 , self.flute_3)

        self.pred_dif_1 = 0
        self.pred_dif_2 = 0
        self.pred_dif_3 = 0
        self.pred_dif_max = 0
        
        #Phm Score
        self.score = 0 

    # Constructing Observations From Environment States
    def _get_obs(self):

        # Παίρνω τα time domain features του current (δηλαδή του αντίστοιχου cut)
        frame = self.df.loc[self.current_step, time_domain_features_columns].to_numpy(dtype=np.float64)

        # Φτιαχνω το observation
        obs = frame

        return obs

    def _get_info(self):
        return {
            "cut": self.current_step,
            "score": self.score,
        }

    def _take_action(self, action):

        # Action Space Example : array([0.79758393, 0.41074004, 0.0026002 ])

        ## Σε κάθε state Sn κάνουμε Predict βάση της προηγούμενης πρόβλεψης Sn-1
        # Φθορα στο μυτακι_1(ν) = φθορα στο μυτακι_1_πρβλεπομενη(ν-1) + prediction του agent
        # Prediction του agent = 100% max diff φθορών σε καταστασεις χρησεις του μηχανηματος (ν -> ν+1)
        
        self.flute_1_pred += (action[0] * MAX_DIFF)
        self.flute_2_pred += (action[1] * MAX_DIFF)
        self.flute_3_pred += (action[2] * MAX_DIFF)
        self.flute_max_pred = max(self.flute_1_pred , self.flute_2_pred , self.flute_3_pred)

        self.flute_1 = self.df.loc[self.current_step, "flute_1"]
        self.flute_2 = self.df.loc[self.current_step, "flute_2"]
        self.flute_3 = self.df.loc[self.current_step, "flute_3"]
        self.flute_max = self.df.loc[self.current_step, "flute_max"]

        # Η διαφορά μεταξύ της προβλεπόμενης τιμής και της κανονικής τιμής
        self.pred_dif_1 = self.flute_1_pred - self.flute_1
        self.pred_dif_2 = self.flute_1_pred - self.flute_1
        self.pred_dif_3 = self.flute_1_pred - self.flute_1
        self.pred_dif_max = self.flute_max_pred - self.flute_max

    def reset(self, seed = None, options = None):
        #We need the following line to seed self.np_random
        super().reset(seed=seed)

        # Επιστρέφω το περιβάλλον στην αρχική του κατάσταση (state s0)
        self._state_s0()

        # Set the current step to the start of the dataset
        self.current_step = 0

        observation = self._get_obs()

        info = self._get_info()

        return observation,info

    def score_function(self,delta):
        if delta < 0:
            return 1 - np.exp(-delta/10)
        else:
            return  1 - np.exp(delta/4.5)
        

    def score_function_original(self, delta):
        if delta < 0:
            return np.exp(-delta/10) - 1
        else:
            return np.exp(delta/4.5) - 1

    def step(self, action):
        # Execute one time step within the environment
        self._take_action(action)

        self.current_step += 1

        reward_flute_1 = self.score_function(self.pred_dif_1)
        reward_flute_2 = self.score_function(self.pred_dif_2)
        reward_flute_3 = self.score_function(self.pred_dif_3)
        reward_flute_max = self.score_function(self.pred_dif_max)

        reward = (REW_WEIGHT[0]*reward_flute_1) + (REW_WEIGHT[1]*reward_flute_2) + (REW_WEIGHT[2]*reward_flute_3) + (REW_WEIGHT[3]*reward_flute_max)

        # H βαθμολογία που θα είχε στο διαγωνισμό
        #self.score -= reward_flute_max
        
        # H βαθμολογία που θα είχε στο διαγωνισμό
        self.score += self.score_function_original(self.pred_dif_max)
        
        
        info = self._get_info()
        #print("In the env self.score is : ", self.score)
        #print(info)
        
        # Ελέγχουμε εάν πάμε να βγούμε εκτός του dataset
        terminated = False
        if self.current_step > MAX_STEPS - 2:
            # τότε πρέπει να κάνουμε reset)
            print("Reached 314", print(info))
            info = self._get_info()

            terminated = True  #διαλέγω terminated καθώς είναι φυσιολογική η λήξη

        observation = self._get_obs()

        #Εδω μπορούμε να βάλουμε κάποιο όριο απόκλισης τιμών στις προβλέψεις ώστε να κάνουμε την μάθηση ταχύτερη
        trunctated = False # Ελέγχουμε εάν υπάρχει κάποιο condition εξωτερικό του περιβάλλοντος πχ. Όριο χρόνου
        if reward < -200000000 : #200 million
            trunctated = True

        return observation, reward, terminated,trunctated, info

    metadata = {"render_modes": ["human"]} #, "rgb_array"], "render_fps": 4}

    def render(self, mode='human'):
        # Render the environment (optional)
        pass

    def close(self):
        # Clean up resources (optional)
        pass

In [ ]:
class CNC_Env_Non_Corrective_with_Delay(gymnasium.Env):

    def __init__(self, df):

        self.df = df
        self.reward_range = (np.NINF , 1 ) # Πρέπει να ορίσω το εύρος των rewards

        # Define the low and high bounds for each dimension of the action space
        low_bound = np.array([0, 0, 0], dtype=np.float64)
        high_bound = np.array([1, 1, 1], dtype=np.float64)

        # Action Space
        self.action_space = gymnasium.spaces.Box(low=low_bound, high=high_bound, dtype=np.float64)

        # Observation Space | Είναι κανονικοποιημένο από 0-1 | 7 sensors * 6 time domain features = 42
        self.observation_space = gymnasium.spaces.Box(low=0, high=1, shape=(42,), dtype=np.float64)

        # Ξεκινάω το περιβάλλον στην αρχική του κατάσταση (state s0)
        self._state_s0()

    def _state_s0(self):
        # βάλε randomness εδώ
        self.current_step = 0
        # flute_1 = df[] diff bettwen d[1] - d[0] * (random val (0,2))
        # state_0 - ( state_1 - state_0)
        self.flute_1 = (2*self.df.loc[0, "flute_1"]) - self.df.loc[1, "flute_1"]
        self.flute_2 = (2*self.df.loc[0, "flute_2"]) - self.df.loc[1, "flute_2"]
        self.flute_3 =  (2*self.df.loc[0, "flute_3"]) - self.df.loc[1, "flute_3"]
        self.flute_max = (2*self.df.loc[0, "flute_max"]) - self.df.loc[1, "flute_max"]

        # Θέτουμε και τα predictions στην αρχική κατάσταση του συστήματος
        self.flute_1_pred = self.flute_1
        self.flute_2_pred = self.flute_2
        self.flute_3_pred = self.flute_3
        self.flute_max_pred = max(self.flute_1 , self.flute_2 , self.flute_3)

        self.pred_dif_1 = 0
        self.pred_dif_2 = 0
        self.pred_dif_3 = 0
        self.pred_dif_max = 0
        
        #Phm Score
        self.score = 0 

    # Constructing Observations From Environment States
    def _get_obs(self):

        # Παίρνω τα time domain features του current (δηλαδή του αντίστοιχου cut)
        frame = self.df.loc[self.current_step, time_domain_features_columns].to_numpy(dtype=np.float64)

        # Φτιαχνω το observation
        obs = frame

        return obs

    def _get_info(self):
        return {
            "cut": self.current_step,
            "score": self.score
        }

    def _take_action(self, action):

        # Action Space Example : array([0.79758393, 0.41074004, 0.0026002 ])

        ## Σε κάθε state Sn κάνουμε Predict βάση της προηγούμενης πρόβλεψης Sn-1
        # Φθορα στο μυτακι_1(ν) = φθορα στο μυτακι_1_πρβλεπομενη(ν-1) + prediction του agent
        # Prediction του agent = 100% max diff φθορών σε καταστασεις χρησεις του μηχανηματος (ν -> ν+1)
        
        self.flute_1_pred += (action[0] * MAX_DIFF)
        self.flute_2_pred += (action[1] * MAX_DIFF)
        self.flute_3_pred += (action[2] * MAX_DIFF)
        self.flute_max_pred = max(self.flute_1_pred , self.flute_2_pred , self.flute_3_pred)

        self.flute_1 = self.df.loc[self.current_step, "flute_1"]
        self.flute_2 = self.df.loc[self.current_step, "flute_2"]
        self.flute_3 = self.df.loc[self.current_step, "flute_3"]
        self.flute_max = self.df.loc[self.current_step, "flute_max"]

        # Η διαφορά μεταξύ της προβλεπόμενης τιμής και της κανονικής τιμής
        self.pred_dif_1 = self.flute_1_pred - self.flute_1
        self.pred_dif_2 = self.flute_1_pred - self.flute_1
        self.pred_dif_3 = self.flute_1_pred - self.flute_1
        self.pred_dif_max = self.flute_max_pred - self.flute_max

    def reset(self, seed = None, options = None):
        #We need the following line to seed self.np_random
        super().reset(seed=seed)

        # Επιστρέφω το περιβάλλον στην αρχική του κατάσταση (state s0)
        self._state_s0()

        # Set the current step to the start of the dataset
        self.current_step = 0

        observation = self._get_obs()
        info = self._get_info()

        return observation,info

    def score_function(self,delta):
        if delta < 0:
            return 1 - np.exp(-delta/10)
        else:
            return  1 - np.exp(delta/4.5)
        
    def score_function_original(self, delta):
        if delta < 0:
            return np.exp(-delta/10) - 1
        else:
            return np.exp(delta/4.5) - 1

    def step(self, action):
        # Execute one time step within the environment
        self._take_action(action)

        self.current_step += 1

        reward_flute_1 = self.score_function(self.pred_dif_1)
        reward_flute_2 = self.score_function(self.pred_dif_2)
        reward_flute_3 = self.score_function(self.pred_dif_3)
        reward_flute_max = self.score_function(self.pred_dif_max) #PHM 

        reward_without_delay = (REW_WEIGHT[0]*reward_flute_1) + (REW_WEIGHT[1]*reward_flute_2) + (REW_WEIGHT[2]*reward_flute_3) + (REW_WEIGHT[3]*reward_flute_max)
        # Παραμετροποιεί το reward έτσι ώστε να ενισχύει τις δράσεις που προσφέρουν οφέλοι σε μέλλοντα χρόνο
        delay_modifier = (self.current_step / (MAX_STEPS-2))
        reward = reward_without_delay * delay_modifier 
        
        # H βαθμολογία που θα είχε στο διαγωνισμό
        #self.score -= reward_flute_max
        
        self.score += self.score_function_original(self.pred_dif_max)
    
        info = self._get_info()
    
        # Ελέγχουμε εάν πάμε να βγούμε εκτός του dataset
        terminated = False
        if self.current_step > MAX_STEPS - 2:
            # τότε πρέπει να κάνουμε reset)
            terminated = True  #διαλέγω terminated καθώς είναι φυσιολογική η λήξη

        observation = self._get_obs()

        #Εδω μπορούμε να βάλουμε κάποιο όριο απόκλισης τιμών στις προβλέψεις ώστε να κάνουμε την μάθηση ταχύτερη
        trunctated = False # Ελέγχουμε εάν υπάρχει κάποιο condition εξωτερικό του περιβάλλοντος πχ. Όριο χρόνου
        if reward < -200000000 : #200milion
            trunctated = True

        return observation, reward, terminated,trunctated, info

    metadata = {"render_modes": ["human"]} #, "rgb_array"], "render_fps": 4}

    def render(self, mode='human'):
        # Render the environment (optional)
        pass

    def close(self):
        # Clean up resources (optional)
        pass

In [ ]:
# Create an instance of the environment with your dataframe
c1_non_corrective_pred_no_delay_env = CNC_Env_Non_Corrective_No_Delay(df)
c1_non_corrective_pred_with_delay_env = CNC_Env_Non_Corrective_with_Delay(df)

c1_corrective_pred_no_delay_env = CNC_Env_Corrective_Pred_No_Delay(df)
c1_corrective_pred_with_delay_env = CNC_Env_Corrective_Pred_with_Delay(df)

In [ ]:
check_env(c1_non_corrective_pred_no_delay_env, warn=True)
check_env(c1_non_corrective_pred_with_delay_env, warn=True)
check_env(c1_corrective_pred_no_delay_env, warn=True)
check_env(c1_corrective_pred_with_delay_env,warn=True)

In [ ]:
# Define the log path
#log_path = os.path.join('training', 'logs')

# Create log directory if it doesn't exist
#os.makedirs(log_path, exist_ok=True)

# Wrap the environment with the Monitor
#non_corrective_pred_env = Monitor(non_corrective_pred_env, log_path)

# Wrap the environment with the Monitor
#corrective_pred_env = Monitor(corrective_pred_env, log_path)

In [ ]:
# Training Help
TIMESTEPS = 10000

# Create a log path for our training logs 
logdir = "logs"
if not os.path.exists(logdir):
    os.makedirs(logdir)

### Tensorboard Command
C:\path >tensorboard --logdir=logs 

## Helper Functions

In [ ]:
# Function to find the highest 'i' value from existing model files
def find_highest_i(models_dir):
    highest_i = 0
    for filename in os.listdir(models_dir):
        if filename.endswith(".zip"):  # Ensure we're only looking at model files
            try:
                # Extract 'i' from the filename based on the TIMESTEPS*i pattern
                i = int(filename.replace('.zip', '').split('/')[-1]) // TIMESTEPS
                if i > highest_i:
                    highest_i = i
            except ValueError:
                # In case the filename doesn't follow the expected pattern
                continue
    return highest_i

In [ ]:
def make_filename(models_dir): 
    if not os.path.exists(models_dir): 
        os.makedirs(models_dir)

    highest_i = find_highest_i(models_dir)
    return f"{models_dir}/{TIMESTEPS*highest_i}.zip"

### **Callback**

In [ ]:
from stable_baselines3.common.callbacks import BaseCallback
import warnings

# Suppress all warnings
warnings.filterwarnings("ignore")

# Define a custom callback to log the score during training
class ScoreLoggerCallback(BaseCallback):
    def __init__(self, verbose=0):
        super(ScoreLoggerCallback, self).__init__(verbose) 
        self.max_score = 0  # Define max_score attribute here

    def _on_step(self) -> bool:
        # Retrieve the wrapped environment from the model
        env = self.model.env.envs[0]  # Assuming you only have one environment in the DummyVecEnv
        
        # Log the score after each training step
        phm_score = env.score
          
        current_step = env.current_step 
        if current_step < (MAX_STEPS - 5):
            # Log the maximum score achieved so far when current step is less than MAX_STEPS - 5
            self.logger.record("rollout/phm_score", self.max_score)
            #print("Current Step is :", current_step)
            #print("The score LESS THAN 315 is :", phm_score)
        
        if current_step >= (MAX_STEPS - 2):
            # Log the score and update max_score when current step is greater than or equal to MAX_STEPS - 2
            self.max_score = phm_score  # Update max_score
            self.logger.record("rollout/phm_score", phm_score)
            #print("Current Step is :", current_step)
            #print("The score at 315 is :", phm_score)
            
        return True

### **Load Model**

In [ ]:
def load_model(model_filename, algo, env):
    if Path(model_filename).exists():
        algo_class = getattr(stable_baselines3, algo)
        loaded_model = algo_class.load(model_filename, env)
        print(f"Successfully loaded model from {model_filename}")
        return loaded_model 
    else:
        print(f"Model file '{model_filename}' does not exist.")

### **Run Model**

In [ ]:
def run_model(env, algo, log_name,  models_dir, max_loops , max_time, first_run, model=None):
    
    callback = ScoreLoggerCallback()
    model_filename = make_filename(models_dir)
    highest_i = find_highest_i(models_dir)
    i = highest_i + 1
    di = i 
    start_time = time.time()
    
    while True:
    
        model_filename = make_filename(models_dir)
    
        if first_run : 
            model.learn(total_timesteps=TIMESTEPS, reset_num_timesteps=False, tb_log_name=log_name, callback=callback)
            model_filename = f"{models_dir}/{TIMESTEPS*i}.zip"
            model.save(model_filename)
            i+1
            first_run = False
            
        model = load_model(model_filename, algo, env)
        model.learn(total_timesteps=TIMESTEPS, reset_num_timesteps=False, tb_log_name=log_name, callback=callback)
        
        model_filename = f"{models_dir}/{TIMESTEPS*i}.zip"
        model.save(model_filename)

        reward, _= evaluate_policy(model, env, n_eval_episodes=2, render=False)
        print(reward)
        i += 1
        
        #trackers
        diff_timesteps = i - di
        print("diff_timesteps is ", diff_timesteps)
        elapsed_time = time.time() - start_time
        print("Elasped time is (sec)",elapsed_time )
        if diff_timesteps > max_loops or elapsed_time > max_time :
            break
        

**Warnings Supressor**

In [ ]:
# Reset warnings to default state
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
#warnings.resetwarnings()

# Algorithms

#### Create the models
We run it only once, after that we load each model

In [ ]:
## -- PPO -- ##
#ppo_model_non_corr_no_delay = PPO("MlpPolicy", c1_non_corrective_pred_no_delay_env, verbose=1, tensorboard_log = logdir)
#ppo_model_non_corr_with_delay = PPO("MlpPolicy", c1_non_corrective_pred_with_delay_env, verbose=1, tensorboard_log = logdir)
#---------------------
#ppo_model_corr_no_delay = PPO("MlpPolicy", c1_corrective_pred_no_delay_env, verbose=1, tensorboard_log = logdir)
#ppo_model_corr_with_delay = PPO("MlpPolicy", c1_corrective_pred_with_delay_env, verbose=1, tensorboard_log = logdir)
#---------------------

## -- A2C -- ##
#a2c_model_non_corr_no_delay = A2C("MlpPolicy", c1_non_corrective_pred_no_delay_env, verbose=1, tensorboard_log = logdir)
#a2c_model_non_corr_with_delay = A2C("MlpPolicy", c1_non_corrective_pred_with_delay_env, verbose=1, tensorboard_log = logdir)
#---------------------
#a2c_model_corr_no_delay = A2C("MlpPolicy", c1_corrective_pred_no_delay_env, verbose=1, tensorboard_log = logdir)
#a2c_model_corr_with_delay = A2C("MlpPolicy", c1_corrective_pred_with_delay_env, verbose=1, tensorboard_log = logdir)

## -- DDPG -- ##
#ddpg_model_non_corr_no_delay = DDPG("MlpPolicy", c1_non_corrective_pred_no_delay_env, verbose=1, tensorboard_log = logdir)
#ddpg_model_non_corr_with_delay = DDPG("MlpPolicy", c1_non_corrective_pred_with_delay_env, verbose=1, tensorboard_log = logdir)
#---------------------
#ddpg_model_corr_no_delay = DDPG("MlpPolicy", c1_corrective_pred_no_delay_env, verbose=1, tensorboard_log = logdir)
#ddpg_model_corr_with_delay = DDPG("MlpPolicy", c1_corrective_pred_with_delay_env, verbose=1, tensorboard_log = logdir)

## -- SAC -- ##
#sac_model_non_corr_no_delay = SAC("MlpPolicy", c1_non_corrective_pred_no_delay_env, verbose=1, tensorboard_log = logdir)
#sac_model_non_corr_with_delay = SAC("MlpPolicy", c1_non_corrective_pred_with_delay_env, verbose=1, tensorboard_log = logdir)
#---------------------
#sac_model_corr_no_delay = SAC("MlpPolicy", c1_corrective_pred_no_delay_env, verbose=1, tensorboard_log = logdir)
#sac_model_corr_with_delay = SAC("MlpPolicy", c1_corrective_pred_with_delay_env, verbose=1, tensorboard_log = logdir)

# PPO

### PPO Non Corrective No Delay

In [ ]:
# FIRST RUN 
#run_model(c1_non_corrective_pred_no_delay_env, "PPO", "PPO_model_non_corr_no_delay", "models/PPO_non_corr_no_delay", 1, 3600, True, ppo_model_non_corr_no_delay) #1hour   
# -----------
# MAIN RUN 
#run_model(c1_non_corrective_pred_no_delay_env, "PPO", "PPO_model_non_corr_no_delay", "models/PPO_non_corr_no_delay", 10, 3600, False,)    

### PPO Non Corrective With Delay

In [ ]:
# FIRST RUN 
#run_model(c1_non_corrective_pred_with_delay_env, "PPO", "PPO_model_non_corr_with_delay", "models/PPO_non_corr_with_delay", 1, 3600, True, ppo_model_non_corr_with_delay) #1hour   
#-------------
# MAIN RUN 
#run_model(c1_non_corrective_pred_with_delay_env, "PPO", "PPO_model_non_corr_with_delay", "models/PPO_non_corr_with_delay", 1000, 10200, False,)    

### PPO Corrective No Delay

In [ ]:
# FIRST RUN
#run_model(c1_corrective_pred_no_delay_env, "PPO", "PPO_model_corr_no_delay", "models/PPO_corr_no_delay", 1, 3600, True, ppo_model_corr_no_delay) #1hour   
#-------------
# MAIN RUN 
#run_model(c1_corrective_pred_no_delay_env, "PPO", "PPO_model_corr_no_delay", "models/PPO_corr_no_delay", 1, 3600, False,)    

### PPO Corrective With Delay

In [ ]:
# FIRST RUN 
#run_model(c1_corrective_pred_with_delay_env, "PPO", "PPO_model_corr_with_delay", "models/PPO_corr_with_delay", 1, 3600, True, ppo_model_corr_with_delay) #1hour   
#-------------
# MAIN RUN 
#run_model(c1_corrective_pred_with_delay_env, "PPO", "PPO_model_corr_with_delay", "models/PPO_corr_with_delay", 100, 3600, False,)    

#### PPO Corrective No Delay (Test 1)

In [ ]:
#models_dir = "models\PPO_non_corr_no_delay_test" 
#ppo_model_non_corr_no_delay_test.learn(total_timesteps=TIMESTEPS, reset_num_timesteps=False, tb_log_name="PPO_non_corr_no_delay_test", callback=callback)

#### PPO Corrective No Delay (Test 2)

In [ ]:
#model_filename = make_filename("models\PPO_non_corr_no_delay_test", TIMESTEPS)
#ppo_corrective_no_delay_test_2 = log_name , model_dir

#ppo_model_non_corr_no_delay_test <- MODEL ,  tb_log_name="PPO_non_corr_no_delay_test2" , models_dir = "models\PPO_non_corr_no_delay_test"

In [ ]:
#run_model(ppo_corrective_no_delay_test_2, c1_non_corrective_pred_no_delay_env, "ppo_model_non_corr_no_delay", "models/a2c_non_corr_no_delay", 0, 3600) #1hour

In [ ]:
# Load PPO_non_corr Model 
#model_filename = make_filename("models\PPO_non_corr_no_delay", TIMESTEPS)
#ppo_model_non_corr_no_delay = PPO.load(model_filename, exp)
#print(f"Successfully loaded model from {model_filename}")

In [ ]:
# Load PPO_non_corr Model TEST 
#model_filename = make_filename("models\PPO_non_corr_no_delay_test", TIMESTEPS)
#ppo_model_non_corr_no_delay_test = PPO.load(model_filename, c1_non_corrective_pred_no_delay_env )
#print(f"Successfully loaded model from {model_filename}")

In [ ]:
# Train PPO_non_corr_no_delay
#models_dir = "models\PPO_non_corr_no_delay_test"
#highest_i = find_highest_i("models\PPO_non_corr_no_delay_test", TIMESTEPS)
# Adjust the range based on the highest 'i' found
#for i in range(highest_i + 1, highest_i + 31):  # Starts from the next available 'i'
#    ppo_model_non_corr_no_delay_test.learn(total_timesteps=TIMESTEPS, reset_num_timesteps=False, tb_log_name="PPO_non_corr_no_delay_test", callback=callback)
#    model_filename = f"{models_dir}/{TIMESTEPS*i}.zip"
#    ppo_model_non_corr_no_delay_test.save(model_filename)

In [ ]:
#model_filename = make_filename("models\PPO_non_corr_no_delay_test", TIMESTEPS)

In [ ]:
#models_dir = "models\PPO_non_corr_no_delay_test"
#i = find_highest_i(models_dir, TIMESTEPS)
#print(i)
#print(TIMESTEPS)
#model_filename = f"{models_dir}/{TIMESTEPS*i}.zip"

#print(model_filename)

In [ ]:
#models_dir = "models\PPO_non_corr_no_delay_test"
#highest_i = find_highest_i(models_dir, TIMESTEPS)
#callback = ScoreLoggerCallback()

#---------

#----------

# Wrap the environment with Monitor and DummyVecEnv
#env = c1_non_corrective_pred_no_delay_env
#env = Monitor(env)
#env = DummyVecEnv([lambda: env])
#ppo_model_non_corr_no_delay = PPO("MlpPolicy", env, verbose=1, tensorboard_log = logdir)
# Adjust the range based on the highest 'i' found
#i = highest_i + 1
#while True:
#    ppo_model_non_corr_no_delay_test.learn(total_timesteps=TIMESTEPS, reset_num_timesteps=False, tb_log_name="PPO_non_corr_no_delay_test2", callback=callback)
#    model_filename = f"{models_dir}/{TIMESTEPS*i}.zip"
#    
#    ppo_model_non_corr_no_delay_test.save(model_filename)
#    
    # Evaluate the model to check if the reward is greater than -4000
    # Assuming you have a function to evaluate the model and return its reward
#    reward, _ = evaluate_policy(ppo_model_non_corr_no_delay, c1_non_corrective_pred_no_delay_env, n_eval_episodes=2, render=False)
#    print(reward)
#    if reward > -4000:
#        break  # Exit the loop if reward exceeds -4000
#    i += 1

# A2C

### A2C Non Corrective No Delay

In [ ]:
# FIRST RUN 
#run_model(c1_non_corrective_pred_no_delay_env, "A2C", "A2C_model_non_corr_no_delay", "models/A2C_non_corr_no_delay", 1, 3600, True, a2c_model_non_corr_no_delay) #1hour   
#-------------
# MAIN RUN 
#run_model(c1_non_corrective_pred_no_delay_env, "A2C", "A2C_model_non_corr_no_delay", "models/A2C_non_corr_no_delay", 100, 3600, False,)    

### A2C Non Corrective With Delay

In [ ]:
# FIRST RUN 
#run_model(c1_non_corrective_pred_no_delay_env, "A2C", "A2C_model_non_corr_with_delay", "models/A2C_non_corr_with_delay", 1, 3600, True, a2c_model_non_corr_with_delay) #1hour   
#-------------
# MAIN RUN 
#run_model(c1_non_corrective_pred_with_delay_env, "A2C", "A2C_model_non_corr_with_delay", "models/A2C_non_corr_with_delay", 1200, 8600, False,)    

### A2C Corrective No Delay

In [ ]:
# FIRST RUN 
#run_model(c1_corrective_pred_no_delay_env, "A2C", "A2C_model_corr_no_delay", "models/A2C_corr_no_delay", 1, 3600, True, a2c_model_corr_no_delay) #1hour   
#-------------
# MAIN RUN 
#run_model(c1_non_corrective_pred_no_delay_env, "A2C", "A2C_model_corr_no_delay", "models/A2C_corr_no_delay", 100, 3600, False,)    

### A2C Corrective With Delay

In [ ]:
# FIRST RUN 
#run_model(c1_corrective_pred_with_delay_env, "A2C", "A2C_model_corr_with_delay", "models/A2C_corr_with_delay", 1, 3600, True, a2c_model_corr_with_delay) #1hour   
#-------------
# MAIN RUN 
#run_model(c1_corrective_pred_with_delay_env, "A2C", "A2C_model_corr_with_delay", "models/A2C_corr_with_delay", 1000, 10600, False,)    

# **DDPG**

### DDPG Non Corrective No Delay

In [ ]:
# FIRST RUN 
#run_model(c1_non_corrective_pred_no_delay_env, "DDPG", "DDPG_model_non_corr_no_delay", "models/DDPG_non_corr_no_delay", 1, 3600, True, ddpg_model_non_corr_no_delay) #1hour   
#-------------
# MAIN RUN 
#run_model(c1_non_corrective_pred_no_delay_env, "DDPG", "DDPG_model_non_corr_no_delay", "models/DDPG_non_corr_no_delay", 100, 4600, False,)    

### DDPG Non Corrective With Delay

In [ ]:
# FIRST RUN 
#run_model(c1_non_corrective_pred_with_delay_env, "DDPG", "DDPG_model_non_corr_with_delay", "models/DDPG_non_corr_with_delay", 1, 3600, True, ddpg_model_non_corr_with_delay) #1hour   
#-------------
# MAIN RUN 
#run_model(c1_non_corrective_pred_with_delay_env, "DDPG", "DDPG_model_non_corr_with_delay", "models/DDPG_non_corr_with_delay", 100, 4600, False,)    

### DDPG Corrective No Delay

In [ ]:
# FIRST RUN 
#run_model(c1_corrective_pred_no_delay_env, "DDPG", "DDPG_model_corr_no_delay", "models/DDPG_corr_no_delay", 1, 3600, True, ddpg_model_corr_no_delay) #1hour   
#-------------
# MAIN RUN 
#run_model(c1_corrective_pred_no_delay_env, "DDPG", "DDPG_model_corr_no_delay", "models/DDPG_corr_no_delay", 1000, 10600, False,)    

### DDPG Corrective With Delay

In [ ]:
# FIRST RUN 
#run_model(c1_corrective_pred_with_delay_env, "DDPG", "DDPG_model_corr_with_delay", "models/DDPG_corr_with_delay", 1, 3600, True, ddpg_model_corr_with_delay) #1hour   
#-------------
# MAIN RUN 
#run_model(c1_corrective_pred_with_delay_env, "DDPG", "DDPG_model_corr_with_delay", "models/DDPG_corr_with_delay", 1000, 4000, False,)    

# **SAC**

### SAC Non Corrective Νo Delay

In [73]:
# FIRST RUN 
#run_model(c1_non_corrective_pred_no_delay_env, "SAC", "SAC_model_non_corr_no_delay", "models/SAC_non_corr_no_delay", 1, 3600, True, sac_model_non_corr_no_delay) #1hour   
#-------------
# MAIN RUN 
run_model(c1_non_corrective_pred_no_delay_env, "SAC", "SAC_model_non_corr_no_delay", "models/SAC_non_corr_no_delay", 1000, 6600, False,)    

Wrapping the env in a DummyVecEnv.
Successfully loaded model from models/SAC_non_corr_no_delay/3600000.zip
Logging to logs\SAC_model_non_corr_no_delay_0
----------------------------------
| rollout/           |           |
|    ep_len_mean     | 33.2      |
|    ep_rew_mean     | -5.59e+08 |
|    phm_score       | 0         |
| time/              |           |
|    episodes        | 90920     |
|    fps             | 35        |
|    time_elapsed    | 2         |
|    total_timesteps | 3610099   |
| train/             |           |
|    actor_loss      | 1.66e+10  |
|    critic_loss     | 1.21e+15  |
|    ent_coef        | 5.86e+06  |
|    ent_coef_loss   | -1.98     |
|    learning_rate   | 0.0003    |
|    n_updates       | 3609998   |
----------------------------------
----------------------------------
| rollout/           |           |
|    ep_len_mean     | 33.2      |
|    ep_rew_mean     | -5.54e+08 |
|    phm_score       | 0         |
| time/              |           |
|    ep

----------------------------------
| rollout/           |           |
|    ep_len_mean     | 33.3      |
|    ep_rew_mean     | -5.31e+08 |
|    phm_score       | 0         |
| time/              |           |
|    episodes        | 90972     |
|    fps             | 37        |
|    time_elapsed    | 49        |
|    total_timesteps | 3611840   |
| train/             |           |
|    actor_loss      | 1.68e+10  |
|    critic_loss     | 1.53e+15  |
|    ent_coef        | 5.23e+06  |
|    ent_coef_loss   | 1.8       |
|    learning_rate   | 0.0003    |
|    n_updates       | 3611739   |
----------------------------------
----------------------------------
| rollout/           |           |
|    ep_len_mean     | 33.3      |
|    ep_rew_mean     | -5.29e+08 |
|    phm_score       | 0         |
| time/              |           |
|    episodes        | 90976     |
|    fps             | 37        |
|    time_elapsed    | 53        |
|    total_timesteps | 3611977   |
| train/            

----------------------------------
| rollout/           |           |
|    ep_len_mean     | 34        |
|    ep_rew_mean     | -5.85e+08 |
|    phm_score       | 0         |
| time/              |           |
|    episodes        | 91028     |
|    fps             | 37        |
|    time_elapsed    | 101       |
|    total_timesteps | 3613758   |
| train/             |           |
|    actor_loss      | 1.69e+10  |
|    critic_loss     | 1e+15     |
|    ent_coef        | 5.19e+06  |
|    ent_coef_loss   | 0.57      |
|    learning_rate   | 0.0003    |
|    n_updates       | 3613657   |
----------------------------------
----------------------------------
| rollout/           |           |
|    ep_len_mean     | 34        |
|    ep_rew_mean     | -5.84e+08 |
|    phm_score       | 0         |
| time/              |           |
|    episodes        | 91032     |
|    fps             | 37        |
|    time_elapsed    | 105       |
|    total_timesteps | 3613895   |
| train/            

----------------------------------
| rollout/           |           |
|    ep_len_mean     | 34.5      |
|    ep_rew_mean     | -5.94e+08 |
|    phm_score       | 0         |
| time/              |           |
|    episodes        | 91084     |
|    fps             | 36        |
|    time_elapsed    | 153       |
|    total_timesteps | 3615692   |
| train/             |           |
|    actor_loss      | 1.69e+10  |
|    critic_loss     | 1.13e+15  |
|    ent_coef        | 5.35e+06  |
|    ent_coef_loss   | 1.02      |
|    learning_rate   | 0.0003    |
|    n_updates       | 3615591   |
----------------------------------
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 34.5     |
|    ep_rew_mean     | -5.9e+08 |
|    phm_score       | 0        |
| time/              |          |
|    episodes        | 91088    |
|    fps             | 36       |
|    time_elapsed    | 157      |
|    total_timesteps | 3615831  |
| train/             |        

----------------------------------
| rollout/           |           |
|    ep_len_mean     | 34.5      |
|    ep_rew_mean     | -5.75e+08 |
|    phm_score       | 0         |
| time/              |           |
|    episodes        | 91140     |
|    fps             | 36        |
|    time_elapsed    | 206       |
|    total_timesteps | 3617618   |
| train/             |           |
|    actor_loss      | 1.69e+10  |
|    critic_loss     | 3.57e+15  |
|    ent_coef        | 5.28e+06  |
|    ent_coef_loss   | -3.78     |
|    learning_rate   | 0.0003    |
|    n_updates       | 3617517   |
----------------------------------
----------------------------------
| rollout/           |           |
|    ep_len_mean     | 34.5      |
|    ep_rew_mean     | -5.73e+08 |
|    phm_score       | 0         |
| time/              |           |
|    episodes        | 91144     |
|    fps             | 36        |
|    time_elapsed    | 210       |
|    total_timesteps | 3617756   |
| train/            

...

----------------------------------
| rollout/           |           |
|    ep_len_mean     | 36        |
|    ep_rew_mean     | -6.17e+08 |
|    phm_score       | 0         |
| time/              |           |
|    episodes        | 96380     |
|    fps             | 32        |
|    time_elapsed    | 13        |
|    total_timesteps | 3810431   |
| train/             |           |
|    actor_loss      | 2.53e+10  |
|    critic_loss     | 7.68e+15  |
|    ent_coef        | 8.79e+06  |
|    ent_coef_loss   | 1.34      |
|    learning_rate   | 0.0003    |
|    n_updates       | 3810330   |
----------------------------------
----------------------------------
| rollout/           |           |
|    ep_len_mean     | 36        |
|    ep_rew_mean     | -6.04e+08 |
|    phm_score       | 0         |
| time/              |           |
|    episodes        | 96384     |
|    fps             | 32        |
|    time_elapsed    | 17        |
|    total_timesteps | 3810573   |
| train/            

----------------------------------
| rollout/           |           |
|    ep_len_mean     | 35.9      |
|    ep_rew_mean     | -6.11e+08 |
|    phm_score       | 0         |
| time/              |           |
|    episodes        | 96436     |
|    fps             | 1         |
|    time_elapsed    | 1935      |
|    total_timesteps | 3812432   |
| train/             |           |
|    actor_loss      | 2.56e+10  |
|    critic_loss     | 8.27e+15  |
|    ent_coef        | 8.87e+06  |
|    ent_coef_loss   | 2.13      |
|    learning_rate   | 0.0003    |
|    n_updates       | 3812331   |
----------------------------------
----------------------------------
| rollout/           |           |
|    ep_len_mean     | 35.8      |
|    ep_rew_mean     | -6.03e+08 |
|    phm_score       | 0         |
| time/              |           |
|    episodes        | 96440     |
|    fps             | 1         |
|    time_elapsed    | 1941      |
|    total_timesteps | 3812576   |
| train/            

----------------------------------
| rollout/           |           |
|    ep_len_mean     | 35.7      |
|    ep_rew_mean     | -6.18e+08 |
|    phm_score       | 0         |
| time/              |           |
|    episodes        | 96492     |
|    fps             | 2         |
|    time_elapsed    | 2002      |
|    total_timesteps | 3814431   |
| train/             |           |
|    actor_loss      | 2.58e+10  |
|    critic_loss     | 7.64e+15  |
|    ent_coef        | 8.76e+06  |
|    ent_coef_loss   | -6.98     |
|    learning_rate   | 0.0003    |
|    n_updates       | 3814330   |
----------------------------------
----------------------------------
| rollout/           |           |
|    ep_len_mean     | 35.7      |
|    ep_rew_mean     | -6.14e+08 |
|    phm_score       | 0         |
| time/              |           |
|    episodes        | 96496     |
|    fps             | 2         |
|    time_elapsed    | 2007      |
|    total_timesteps | 3814573   |
| train/            

----------------------------------
| rollout/           |           |
|    ep_len_mean     | 35.5      |
|    ep_rew_mean     | -6.04e+08 |
|    phm_score       | 0         |
| time/              |           |
|    episodes        | 96548     |
|    fps             | 3         |
|    time_elapsed    | 2065      |
|    total_timesteps | 3816418   |
| train/             |           |
|    actor_loss      | 2.59e+10  |
|    critic_loss     | 9.16e+15  |
|    ent_coef        | 9.05e+06  |
|    ent_coef_loss   | -2.06     |
|    learning_rate   | 0.0003    |
|    n_updates       | 3816317   |
----------------------------------
----------------------------------
| rollout/           |           |
|    ep_len_mean     | 35.5      |
|    ep_rew_mean     | -6.11e+08 |
|    phm_score       | 0         |
| time/              |           |
|    episodes        | 96552     |
|    fps             | 3         |
|    time_elapsed    | 2069      |
|    total_timesteps | 3816560   |
| train/            

----------------------------------
| rollout/           |           |
|    ep_len_mean     | 35.4      |
|    ep_rew_mean     | -6.02e+08 |
|    phm_score       | 0         |
| time/              |           |
|    episodes        | 96604     |
|    fps             | 3         |
|    time_elapsed    | 2127      |
|    total_timesteps | 3818400   |
| train/             |           |
|    actor_loss      | 2.63e+10  |
|    critic_loss     | 9.41e+15  |
|    ent_coef        | 9.3e+06   |
|    ent_coef_loss   | -0.0198   |
|    learning_rate   | 0.0003    |
|    n_updates       | 3818299   |
----------------------------------
----------------------------------
| rollout/           |           |
|    ep_len_mean     | 35.4      |
|    ep_rew_mean     | -6.03e+08 |
|    phm_score       | 0         |
| time/              |           |
|    episodes        | 96608     |
|    fps             | 4         |
|    time_elapsed    | 2132      |
|    total_timesteps | 3818541   |
| train/            

### SAC Non Corrective With Delay

In [74]:
# FIRST RUN 
#run_model(c1_non_corrective_pred_with_delay_env, "SAC", "SAC_model_non_corr_with_delay", "models/SAC_non_corr_with_delay", 1, 3600, True, sac_model_non_corr_with_delay) #1hour   
#-------------
# MAIN RUN 
run_model(c1_non_corrective_pred_with_delay_env, "SAC", "SAC_model_non_corr_with_delay", "models/SAC_non_corr_with_delay", 10000, 12600, False,)   

Wrapping the env in a DummyVecEnv.
Successfully loaded model from models/SAC_non_corr_with_delay/2240000.zip
Logging to logs\SAC_model_non_corr_with_delay_0
----------------------------------
| rollout/           |           |
|    ep_len_mean     | 27.4      |
|    ep_rew_mean     | -5.65e+08 |
|    phm_score       | 0         |
| time/              |           |
|    episodes        | 72140     |
|    fps             | 35        |
|    time_elapsed    | 3         |
|    total_timesteps | 2250109   |
| train/             |           |
|    actor_loss      | 6.49e+10  |
|    critic_loss     | 1.55e+16  |
|    ent_coef        | 2.01e+07  |
|    ent_coef_loss   | 3.89      |
|    learning_rate   | 0.0003    |
|    n_updates       | 2250008   |
----------------------------------
----------------------------------
| rollout/           |           |
|    ep_len_mean     | 27.4      |
|    ep_rew_mean     | -5.79e+08 |
|    phm_score       | 0         |
| time/              |           |
|  

----------------------------------
| rollout/           |           |
|    ep_len_mean     | 27.4      |
|    ep_rew_mean     | -5.27e+08 |
|    phm_score       | 0         |
| time/              |           |
|    episodes        | 72192     |
|    fps             | 32        |
|    time_elapsed    | 47        |
|    total_timesteps | 2251528   |
| train/             |           |
|    actor_loss      | 6.59e+10  |
|    critic_loss     | 1.94e+16  |
|    ent_coef        | 2.2e+07   |
|    ent_coef_loss   | 2.06      |
|    learning_rate   | 0.0003    |
|    n_updates       | 2251427   |
----------------------------------
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 27.3     |
|    ep_rew_mean     | -5.2e+08 |
|    phm_score       | 0        |
| time/              |          |
|    episodes        | 72196    |
|    fps             | 32       |
|    time_elapsed    | 50       |
|    total_timesteps | 2251637  |
| train/             |        

----------------------------------
| rollout/           |           |
|    ep_len_mean     | 27.3      |
|    ep_rew_mean     | -4.93e+08 |
|    phm_score       | 0         |
| time/              |           |
|    episodes        | 72248     |
|    fps             | 32        |
|    time_elapsed    | 95        |
|    total_timesteps | 2253059   |
| train/             |           |
|    actor_loss      | 6.6e+10   |
|    critic_loss     | 2e+16     |
|    ent_coef        | 2.28e+07  |
|    ent_coef_loss   | 2.02      |
|    learning_rate   | 0.0003    |
|    n_updates       | 2252958   |
----------------------------------
----------------------------------
| rollout/           |           |
|    ep_len_mean     | 27.3      |
|    ep_rew_mean     | -4.93e+08 |
|    phm_score       | 0         |
| time/              |           |
|    episodes        | 72252     |
|    fps             | 32        |
|    time_elapsed    | 98        |
|    total_timesteps | 2253169   |
| train/            

----------------------------------
| rollout/           |           |
|    ep_len_mean     | 27.4      |
|    ep_rew_mean     | -5.53e+08 |
|    phm_score       | 0         |
| time/              |           |
|    episodes        | 72304     |
|    fps             | 32        |
|    time_elapsed    | 141       |
|    total_timesteps | 2254595   |
| train/             |           |
|    actor_loss      | 6.6e+10   |
|    critic_loss     | 2.02e+16  |
|    ent_coef        | 2.23e+07  |
|    ent_coef_loss   | -3.38     |
|    learning_rate   | 0.0003    |
|    n_updates       | 2254494   |
----------------------------------
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 27.4     |
|    ep_rew_mean     | -5.5e+08 |
|    phm_score       | 0        |
| time/              |          |
|    episodes        | 72308    |
|    fps             | 32       |
|    time_elapsed    | 144      |
|    total_timesteps | 2254704  |
| train/             |        

---------------------------------
| rollout/           |          |
|    ep_len_mean     | 27.4     |
|    ep_rew_mean     | -5.8e+08 |
|    phm_score       | 0        |
| time/              |          |
|    episodes        | 72360    |
|    fps             | 32       |
|    time_elapsed    | 189      |
|    total_timesteps | 2256131  |
| train/             |          |
|    actor_loss      | 6.68e+10 |
|    critic_loss     | 1.95e+16 |
|    ent_coef        | 2.29e+07 |
|    ent_coef_loss   | 0.868    |
|    learning_rate   | 0.0003   |
|    n_updates       | 2256030  |
---------------------------------
----------------------------------
| rollout/           |           |
|    ep_len_mean     | 27.4      |
|    ep_rew_mean     | -5.66e+08 |
|    phm_score       | 0         |
| time/              |           |
|    episodes        | 72364     |
|    fps             | 32        |
|    time_elapsed    | 192       |
|    total_timesteps | 2256240   |
| train/             |           |
|  

...

----------------------------------
| rollout/           |           |
|    ep_len_mean     | 35.2      |
|    ep_rew_mean     | -5.63e+08 |
|    phm_score       | 0         |
| time/              |           |
|    episodes        | 85088     |
|    fps             | 38        |
|    time_elapsed    | 11        |
|    total_timesteps | 2630451   |
| train/             |           |
|    actor_loss      | 4.73e+10  |
|    critic_loss     | 8.33e+16  |
|    ent_coef        | 4.79e+07  |
|    ent_coef_loss   | 3.36      |
|    learning_rate   | 0.0003    |
|    n_updates       | 2630350   |
----------------------------------
----------------------------------
| rollout/           |           |
|    ep_len_mean     | 35.2      |
|    ep_rew_mean     | -5.69e+08 |
|    phm_score       | 0         |
| time/              |           |
|    episodes        | 85092     |
|    fps             | 38        |
|    time_elapsed    | 15        |
|    total_timesteps | 2630592   |
| train/            

----------------------------------
| rollout/           |           |
|    ep_len_mean     | 35.4      |
|    ep_rew_mean     | -5.82e+08 |
|    phm_score       | 0         |
| time/              |           |
|    episodes        | 85144     |
|    fps             | 38        |
|    time_elapsed    | 63        |
|    total_timesteps | 2632451   |
| train/             |           |
|    actor_loss      | 4.7e+10   |
|    critic_loss     | 7.79e+16  |
|    ent_coef        | 4.65e+07  |
|    ent_coef_loss   | 4.51      |
|    learning_rate   | 0.0003    |
|    n_updates       | 2632350   |
----------------------------------
----------------------------------
| rollout/           |           |
|    ep_len_mean     | 35.5      |
|    ep_rew_mean     | -5.79e+08 |
|    phm_score       | 0         |
| time/              |           |
|    episodes        | 85148     |
|    fps             | 38        |
|    time_elapsed    | 67        |
|    total_timesteps | 2632595   |
| train/            

----------------------------------
| rollout/           |           |
|    ep_len_mean     | 35.8      |
|    ep_rew_mean     | -5.64e+08 |
|    phm_score       | 0         |
| time/              |           |
|    episodes        | 85200     |
|    fps             | 38        |
|    time_elapsed    | 115       |
|    total_timesteps | 2634453   |
| train/             |           |
|    actor_loss      | 4.69e+10  |
|    critic_loss     | 1.08e+17  |
|    ent_coef        | 4.71e+07  |
|    ent_coef_loss   | 1.3       |
|    learning_rate   | 0.0003    |
|    n_updates       | 2634352   |
----------------------------------
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 35.8     |
|    ep_rew_mean     | -5.6e+08 |
|    phm_score       | 0        |
| time/              |          |
|    episodes        | 85204    |
|    fps             | 38       |
|    time_elapsed    | 119      |
|    total_timesteps | 2634591  |
| train/             |        

----------------------------------
| rollout/           |           |
|    ep_len_mean     | 35.2      |
|    ep_rew_mean     | -5.61e+08 |
|    phm_score       | 0         |
| time/              |           |
|    episodes        | 85256     |
|    fps             | 38        |
|    time_elapsed    | 166       |
|    total_timesteps | 2636406   |
| train/             |           |
|    actor_loss      | 4.67e+10  |
|    critic_loss     | 8.52e+16  |
|    ent_coef        | 4.74e+07  |
|    ent_coef_loss   | -0.349    |
|    learning_rate   | 0.0003    |
|    n_updates       | 2636305   |
----------------------------------
----------------------------------
| rollout/           |           |
|    ep_len_mean     | 35.3      |
|    ep_rew_mean     | -5.74e+08 |
|    phm_score       | 0         |
| time/              |           |
|    episodes        | 85260     |
|    fps             | 38        |
|    time_elapsed    | 170       |
|    total_timesteps | 2636550   |
| train/            

----------------------------------
| rollout/           |           |
|    ep_len_mean     | 34.8      |
|    ep_rew_mean     | -5.73e+08 |
|    phm_score       | 0         |
| time/              |           |
|    episodes        | 85312     |
|    fps             | 38        |
|    time_elapsed    | 218       |
|    total_timesteps | 2638354   |
| train/             |           |
|    actor_loss      | 4.66e+10  |
|    critic_loss     | 8.2e+16   |
|    ent_coef        | 4.71e+07  |
|    ent_coef_loss   | 0.83      |
|    learning_rate   | 0.0003    |
|    n_updates       | 2638253   |
----------------------------------
----------------------------------
| rollout/           |           |
|    ep_len_mean     | 34.8      |
|    ep_rew_mean     | -5.78e+08 |
|    phm_score       | 0         |
| time/              |           |
|    episodes        | 85316     |
|    fps             | 38        |
|    time_elapsed    | 223       |
|    total_timesteps | 2638496   |
| train/            

### SAC Corrective no Delay

In [ ]:
# FIRST RUN 
#run_model(c1_corrective_pred_no_delay_env, "SAC", "SAC_model_corr_no_delay", "models/SAC_corr_no_delay", 1, 3600, True, sac_model_corr_no_delay) #1hour   
#-------------
# MAIN RUN 
#run_model(c1_corrective_pred_no_delay_env, "SAC", "SAC_model_corr_no_delay", "models/SAC_corr_no_delay", 10, 3600, False,)    

### SAC Corrective with Delay

In [ ]:
# FIRST RUN 
#run_model(c1_corrective_pred_with_delay_env, "SAC", "SAC_model_corr_with_delay", "models/SAC_corr_with_delay", 1, 3600, True, sac_model_corr_with_delay) #1hour   
#-------------
# MAIN RUN 
#run_model(c1_corrective_pred_with_delay_env, "SAC", "SAC_model_corr_with_delay", "models/SAC_corr_with_delay", 10, 3600, False,)    

# Evaluate policy 

Evaluate_policy, returns the mean reward per episode and the standard deviation of the reward per episode

-> The "mean reward per episode" represents the average reward obtained per episode over the evaluation period.
-> The "standard deviation of the reward per episode" indicates how much the rewards vary from the mean reward per episode.

Default Evalueate Policy

In [ ]:
#evaluate_policy(model_xyz , non_corrective_pred_env_c4 , n_eval_episodes=10, render=False)

**Πρεπει να φτιάξω μια συνάρτηση για τα non corrective environment**

To πρόβλημα ποιο είναι ; 

Το πρόβλημα είναι ότι στα non corr env μπορεί να πέσω πάνω σε μοντέλο που δεν τερματίζει οπότε αξίζει να ψάξω 
ποιο τερματίζει και να χρησιμοποιήσω εκείνο. 

Σκέψου αναδρομικά πήγαινε προς τα πίσω μέχρι να βρεις κάποιο το οποίο τερματίζει. (Εάν δεν βρείς σταμάτα)

In [ ]:
# Function to find the highest 'i' value from existing model files
def find_highest_i(models_dir):
    highest_i = 0
    for filename in os.listdir(models_dir):
        if filename.endswith(".zip"):  # Ensure we're only looking at model files
            try:
                # Extract 'i' from the filename based on the TIMESTEPS*i pattern
                i = int(filename.replace('.zip', '').split('/')[-1]) // TIMESTEPS
                if i > highest_i:
                    highest_i = i
            except ValueError:
                # In case the filename doesn't follow the expected pattern
                continue
    return highest_i

#=====================================================

def load_model(model_filename, algo, env):
    if Path(model_filename).exists():
        algo_class = getattr(stable_baselines3, algo)
        loaded_model = algo_class.load(model_filename, env)
        print(f"Successfully loaded model from {model_filename}")
        return loaded_model 
    else:
        print(f"Model file '{model_filename}' does not exist.")

In [ ]:
def make_filename_i(models_dir, i): 
    if not os.path.exists(models_dir): 
        os.makedirs(models_dir)
        
    #highest_i = find_highest_i(models_dir)
    return f"{models_dir}/{TIMESTEPS*i}.zip"

In [ ]:
import os
import sys
from contextlib import redirect_stdout
def find_314_non_corr(algo,  env, model_dir,  n_eval_episodes=1, render=False):
    
    reward_314_log = []
    phm_score_log = []  # keep all the phm_scores
    reward_timestep_log = []
    
    found = False
    reached_314 = False
    i = find_highest_i(model_dir)  #the latest trained model
    while not found :
            
        total_reward = 0.0
        #Load model at timestep i
        model_filename = make_filename_i(model_dir , i)
        #model = load_model(model_filename, algo , env)
        # Για να μην κάνει τόσα print
        with open(os.devnull, 'w') as fnull, redirect_stdout(fnull):
            model = load_model(model_filename, algo, env)
        
        for _ in range(n_eval_episodes):
            obs = env.reset()
            done = False
            episode_reward = 0.0
            phm_score = 0.0

            while not done:

                if render:
                    env.render()
                if isinstance(obs, tuple):  # Check if the observation is a tuple
                    obs = obs[0]  # If it is, extract the first element
                action, _ = model.predict(obs, deterministic=True)

                obs, reward, terminated, trunctated, info = env.step(action)
                episode_reward += reward
                #print(info)

                # Retrieve the cut information from the info dictionary
                current_step = info.get('cut', 0)
                phm_score = info.get('score', 0)
                #score_314 = info.get('score_314',0) # 0 is the default value assigned if the score_314 doesnt exist in the info dict
                
                #print(current_step, "-> " , score_314)
                #print(phm_score)
                #print("Current step is ", current_step, " AND the phm score is : " ,phm_score)

                if current_step > 313 :
                    
                    #print("Phm score is : ", phm_score)
                    #print("Reward is :", episode_reward)
                    
                    phm_score_log.append([phm_score, i])
                    reward_314_log.append([episode_reward, i])
                    reached_314 = True
                    done = True

                if terminated or trunctated : 
                    #print(current_step)
                    #print("Terminated is : ", terminated)
                    #print("Trunctuated is : ", trunctated)
                    done = True

            total_reward += episode_reward
            reward = total_reward / n_eval_episodes
            reward_timestep_log.append([reward, i])
        
        i = i-1
        if(i<=1): 
            if(reached_314 != True) : 
                print("-----------------------------------")
                print("No model has reached the cut '314' ")
                print("-----------------------------------")
            found = True 

    return reward_timestep_log, phm_score_log, reward_314_log

In [ ]:
import matplotlib.pyplot as plt

arr = [1,2,3]

plt.plot(arr)

In [ ]:
def print_results(reward_timestep_log, phm_score_log, reward_314_log) : 
    
    print()
    max_reward, timestep_i = max(reward_timestep_log, key=lambda entry: entry[0])
    print("Max Reward is : ", max_reward, " and it is found at timestep : ", timestep_i)

    if phm_score_log: 
        min_phm, timestep_i = min(phm_score_log, key=lambda entry: entry[0])
        print("Min Phm Score is : ", min_phm, " and it is found at timestep : ", timestep_i)
    
    if reward_314_log:
        max_reward_314, timestep_i =  max(reward_314_log, key=lambda entry: entry[0])
        print("Max Reward-314 is : ", max_reward_314, " and it is found at timestep : ", timestep_i)


In [ ]:
max_reward, timestep_i = max(reward_timestep_log, key=lambda entry: entry[0])
print("Max Reward is : ", max_reward, " and it is found at timestep : ", timestep_i, " *10,000")

In [ ]:
print("OG function ", evaluate_policy(ppo_model_non_corr_no_delay, c6_non_corrective_pred_no_delay_env, n_eval_episodes=10, render=False) )
evaluate_and_print_policy(ppo_model_non_corr_no_delay, c6_non_corrective_pred_no_delay_env, "PPO Non-Corr No Delay")

In [ ]:
df_c1 = pd.read_csv("./normalised_time_domain_features_with_cut_and_flute_max.csv")
df_c4 = pd.read_csv("./normalised_time_domain_features_c4_with_cut_and_flute_max.csv")
df_c6 = pd.read_csv("./normalised_time_domain_features_c6_with_cut_and_flute_max.csv")

In [ ]:
## -- CUTTER 1 -- ##
c1_non_corrective_pred_no_delay_env = Monitor(CNC_Env_Non_Corrective_No_Delay(df_c1), logdir)
c1_non_corrective_pred_with_delay_env = Monitor(CNC_Env_Non_Corrective_with_Delay(df_c1), logdir)
#------------------------
c1_corrective_pred_no_delay_env = Monitor(CNC_Env_Corrective_Pred_No_Delay(df_c1), logdir)
c1_corrective_pred_with_delay_env = Monitor(CNC_Env_Corrective_Pred_with_Delay(df_c1), logdir)

# -- CUTTER 2 -- ## 
c4_non_corrective_pred_no_delay_env = Monitor(CNC_Env_Non_Corrective_No_Delay(df_c4), logdir)
c4_non_corrective_pred_with_delay_env = Monitor(CNC_Env_Non_Corrective_with_Delay(df_c4), logdir)
#------------------------
c4_corrective_pred_no_delay_env = Monitor(CNC_Env_Corrective_Pred_No_Delay(df_c4), logdir)
c4_corrective_pred_with_delay_env = Monitor(CNC_Env_Corrective_Pred_with_Delay(df_c4), logdir)

# -- CUTTER 3 -- ## 
c6_non_corrective_pred_no_delay_env = Monitor(CNC_Env_Non_Corrective_No_Delay(df_c6), logdir)
c6_non_corrective_pred_with_delay_env = Monitor(CNC_Env_Non_Corrective_with_Delay(df_c6), logdir)
#------------------------
c6_corrective_pred_no_delay_env = Monitor(CNC_Env_Corrective_Pred_No_Delay(df_c6), logdir)
c6_corrective_pred_with_delay_env = Monitor(CNC_Env_Corrective_Pred_with_Delay(df_c6), logdir)

In [ ]:
# Loading 

#PPO
model_filename = make_filename("models\PPO_non_corr_no_delay")
ppo_model_non_corr_no_delay = load_model(model_filename, "PPO", c1_non_corrective_pred_no_delay_env)
#-
model_filename = make_filename("models\PPO_non_corr_with_delay")
ppo_model_non_corr_with_delay = load_model(model_filename, "PPO", c1_non_corrective_pred_with_delay_env)
#-
model_filename = make_filename("models\PPO_corr_no_delay")
ppo_model_corr_no_delay = load_model(model_filename, "PPO", c1_corrective_pred_no_delay_env)
#-
model_filename = make_filename("models\PPO_corr_with_delay")
ppo_model_corr_with_delay = load_model(model_filename, "PPO", c1_corrective_pred_with_delay_env)
#------------------
#A2C
model_filename = make_filename("models\A2C_non_corr_no_delay")
a2c_model_non_corr_no_delay = load_model(model_filename, "A2C", c1_non_corrective_pred_no_delay_env) 
#-
model_filename = make_filename("models\A2C_non_corr_with_delay")
a2c_model_non_corr_with_delay = load_model(model_filename, "A2C", c1_non_corrective_pred_with_delay_env)
#-
model_filename = make_filename("models\A2C_corr_no_delay")
a2c_model_corr_no_delay = load_model(model_filename, "A2C", c1_corrective_pred_no_delay_env)
#-
model_filename = make_filename("models\A2C_corr_with_delay")
a2c_model_corr_with_delay = load_model(model_filename, "A2C", c1_corrective_pred_with_delay_env)
#------------------
#DDPG
model_filename = make_filename("models\DDPG_non_corr_no_delay")
ddpg_model_non_corr_no_delay = load_model(model_filename, "DDPG", c1_non_corrective_pred_no_delay_env)
#-
model_filename = make_filename("models\DDPG_non_corr_with_delay")
ddpg_model_non_corr_with_delay = load_model(model_filename, "DDPG", c1_non_corrective_pred_with_delay_env)
#-
model_filename = make_filename("models\DDPG_corr_no_delay")
ddpg_model_corr_no_delay = load_model(model_filename, "DDPG", c1_corrective_pred_no_delay_env)
#-
model_filename = make_filename("models\DDPG_corr_with_delay")
ddpg_model_corr_with_delay = load_model(model_filename, "DDPG", c1_corrective_pred_with_delay_env)
#------------------
#SAC
model_filename = make_filename("models\SAC_non_corr_no_delay")
sac_model_non_corr_no_delay = load_model(model_filename, "SAC", c1_non_corrective_pred_no_delay_env)
#-
model_filename = make_filename("models\SAC_non_corr_with_delay")
sac_model_non_corr_with_delay = load_model(model_filename, "SAC", c1_non_corrective_pred_with_delay_env)
#-
model_filename = make_filename("models\SAC_corr_no_delay")
sac_model_corr_no_delay = load_model(model_filename, "SAC", c1_corrective_pred_no_delay_env)
#-
model_filename = make_filename("models\SAC_corr_with_delay")
sac_model_corr_with_delay = load_model(model_filename, "SAC", c1_corrective_pred_with_delay_env)

In [ ]:
def evaluate_policy_with_score(model, env, n_eval_episodes=10, render=False):
    total_reward = 0.0
    phm_score_log = []  # keep all the phm_scores
    for _ in range(n_eval_episodes):
        obs = env.reset()
        done = False
        episode_reward = 0.0
        phm_score = 0.0
        
        while not done:
            
            if render:
                env.render()
            if isinstance(obs, tuple):  # Check if the observation is a tuple
                obs = obs[0]  # If it is, extract the first element
            action, _ = model.predict(obs, deterministic=True)
            obs, reward, terminated, trunctated, info = env.step(action)
            #print(info)
            episode_reward += reward
            # Retrieve the cut information from the info dictionary
            current_step = info.get('cut', 0)
            phm_score = info.get('score', 0)

            #print(episode_reward)
            print(phm_score) 
            # print(terminated, current_step, trunctated)
            print(current_step)
            if current_step > (MAX_STEPS - 10):
                # Assuming that the PHM score is the reward in this case
                print("hello")
                phm_score = info.get('score', 0)
                print(phm_score)
                phm_score_log.append(phm_score)
                done = True
                
            if terminated or trunctated : 
                done = True

        
        total_reward += episode_reward

    avg_reward = total_reward / n_eval_episodes
    return avg_reward, phm_score_log

In [ ]:
def evaluate_and_print_policy(model, env, model_name):
    avg_reward, avg_phm_score = evaluate_policy_with_score(model, env, n_eval_episodes=1, render=False)
    print(f"{model_name} Evaluation:")
    print("Average Reward:", avg_reward)
    print("PHM Score:", avg_phm_score)
    return avg_reward, avg_phm_score

# Results

r = reward_timestep_log, <br>
p = phm_score_log, <br>
r_314 = reward_314_log <br>

# **PPO**

### PPO Corr

In [ ]:
r,p,r_314 = find_314_non_corr(algo="PPO", env=c4_corrective_pred_no_delay_env, model_dir="models\PPO_corr_no_delay" , )
print('\033[1m  PPO Corrective Env | No Delay | Cutter 4   \033[0m')
print_results(r, p, r_314)

In [ ]:
r,p,r_314 = find_314_non_corr(algo="PPO", env=c6_corrective_pred_no_delay_env, model_dir="models\PPO_corr_no_delay" , )
print('\033[1m  PPO Corrective Env | No Delay | Cutter 6   \033[0m')
print_results(r, p, r_314)

In [ ]:
r,p,r_314 = find_314_non_corr(algo="PPO", env=c4_corrective_pred_with_delay_env, model_dir="models\PPO_corr_with_delay" , )
print('\033[1m  PPO Corrective Env | With Delay | Cutter 4   \033[0m')
print_results(r, p, r_314)

In [ ]:
r,p,r_314 = find_314_non_corr(algo="PPO", env=c6_corrective_pred_with_delay_env, model_dir="models\PPO_corr_with_delay" , )
print('\033[1m  PPO Corrective Env | With Delay | Cutter 6   \033[0m')
print_results(r, p, r_314)

### PPO Non Corr

In [ ]:
r,p,r_314 = find_314_non_corr(algo="PPO", env=c4_non_corrective_pred_no_delay_env, model_dir="models\PPO_non_corr_no_delay" ,)
print('\033[1m  PPO Non Corrective Env | No Delay | Cutter 4   \033[0m')
print_results(r, p, r_314)

In [ ]:
r,p,r_314 = find_314_non_corr(algo="PPO", env=c6_non_corrective_pred_no_delay_env, model_dir="models\PPO_non_corr_no_delay" , )
print('\033[1m  PPO Non Corrective Env | No Delay | Cutter 6   \033[0m')
print_results(r, p, r_314)

In [ ]:
r,p,r_314 = find_314_non_corr(algo="PPO", env=c4_non_corrective_pred_with_delay_env, model_dir="models\PPO_non_corr_with_delay" ,)
print('\033[1m  PPO Non Corrective Env | With Delay | Cutter 4   \033[0m')
print_results(r, p, r_314) 

In [ ]:
r,p,r_314 = find_314_non_corr(algo="PPO", env=c6_non_corrective_pred_with_delay_env, model_dir="models\PPO_non_corr_with_delay" ,)
print('\033[1m  PPO Non Corrective Env | With Delay | Cutter 6   \033[0m')
print_results(r, p, r_314) 

# **A2C**

### A2C Corr

In [ ]:
r,p,r_314 = find_314_non_corr(algo="A2C", env=c4_corrective_pred_no_delay_env, model_dir="models\A2C_corr_no_delay" , )
print('\033[1m  A2C Corrective Env | No Delay | Cutter 4   \033[0m')
print_results(r, p, r_314)

In [ ]:
r,p,r_314 = find_314_non_corr(algo="A2C", env=c6_corrective_pred_no_delay_env, model_dir="models\A2C_corr_no_delay" , )
print('\033[1m  A2C Corrective Env | No Delay | Cutter 6   \033[0m')
print_results(r, p, r_314)

In [ ]:
r,p,r_314 = find_314_non_corr(algo="A2C", env=c4_corrective_pred_with_delay_env, model_dir="models\A2C_corr_with_delay" , )
print('\033[1m  A2C Corrective Env | With Delay | Cutter 4   \033[0m')
print_results(r, p, r_314)

In [ ]:
r,p,r_314 = find_314_non_corr(algo="A2C", env=c6_corrective_pred_with_delay_env, model_dir="models\A2C_corr_with_delay" , )
print('\033[1m  A2C Corrective Env | With Delay | Cutter 6   \033[0m')
print_results(r, p, r_314)

### A2C Non Corr

In [ ]:
r,p,r_314 = find_314_non_corr(algo="A2C", env=c4_non_corrective_pred_no_delay_env, model_dir="models\A2C_non_corr_no_delay" , )
print('\033[1m  A2C Non Corrective Env | No Delay | Cutter 4   \033[0m')
print_results(r, p, r_314)

In [ ]:
r,p,r_314 = find_314_non_corr(algo="A2C", env=c6_non_corrective_pred_no_delay_env, model_dir="models\A2C_non_corr_no_delay" , )
print('\033[1m  A2C Non Corrective Env | No Delay | Cutter 6   \033[0m')
print_results(r, p, r_314)

In [ ]:
r,p,r_314 = find_314_non_corr(algo="A2C", env=c4_non_corrective_pred_with_delay_env, model_dir="models\A2C_non_corr_with_delay" , )
print('\033[1m  A2C Non Corrective Env | With Delay | Cutter 4   \033[0m')
print_results(r, p, r_314)

In [ ]:
r,p,r_314 = find_314_non_corr(algo="A2C", env=c6_non_corrective_pred_with_delay_env, model_dir="models\A2C_non_corr_with_delay" , )
print('\033[1m  A2C Non Corrective Env | With Delay | Cutter 6   \033[0m')
print_results(r, p, r_314)

# **DDPG**

### DDPG Corr

In [ ]:
r,p,r_314 = find_314_non_corr(algo="DDPG", env=c4_corrective_pred_no_delay_env, model_dir="models\DDPG_corr_no_delay" , )
print('\033[1m  DDPG Corrective Env | No Delay | Cutter 4   \033[0m')
print_results(r, p, r_314)

In [ ]:
r,p,r_314 = find_314_non_corr(algo="DDPG", env=c6_corrective_pred_no_delay_env, model_dir="models\DDPG_corr_no_delay" , )
print('\033[1m  DDPG Corrective Env | No Delay | Cutter 6A\033[0m')
print_results(r, p, r_314)

In [ ]:
r,p,r_314 = find_314_non_corr(algo="DDPG", env=c4_corrective_pred_with_delay_env, model_dir="models\DDPG_corr_with_delay" , )
print('\033[1m  DDPG Corrective Env | With Delay | Cutter 4   \033[0m')
print_results(r, p, r_314)

In [ ]:
r,p,r_314 = find_314_non_corr(algo="DDPG", env=c6_corrective_pred_with_delay_env, model_dir="models\DDPG_corr_with_delay" , )
print('\033[1m  DDPG Corrective Env | With Delay | Cutter 6   \033[0m')
print_results(r, p, r_314)

### DDPG Non Corr

In [ ]:
r,p,r_314 = find_314_non_corr(algo="DDPG", env=c4_non_corrective_pred_no_delay_env, model_dir="models\DDPG_non_corr_no_delay" , )
print('\033[1m  DDPG Non Corrective Env | No Delay | Cutter 4   \033[0m')
print_results(r, p, r_314)

In [ ]:
r,p,r_314 = find_314_non_corr(algo="DDPG", env=c6_non_corrective_pred_no_delay_env, model_dir="models\DDPG_non_corr_no_delay" , )
print('\033[1m  DDPG Non Corrective Env | No Delay | Cutter 6   \033[0m')
print_results(r, p, r_314)

In [ ]:
r,p,r_314 = find_314_non_corr(algo="DDPG", env=c4_non_corrective_pred_with_delay_env, model_dir="models\DDPG_non_corr_with_delay" , )
print('\033[1m  DDPG Non Corrective Env | With Delay | Cutter 4   \033[0m')
print_results(r, p, r_314) 

In [ ]:
r,p,r_314 = find_314_non_corr(algo="DDPG", env=c6_non_corrective_pred_with_delay_env, model_dir="models\DDPG_non_corr_with_delay" , )
print('\033[1m  DDPG Non Corrective Env | With Delay | Cutter 6   \033[0m')
print_results(r, p, r_314) 

# **SAC**

### SAC Corr

In [ ]:
r,p,r_314 = find_314_non_corr(algo="SAC", env=c4_corrective_pred_no_delay_env, model_dir="models\SAC_corr_no_delay" , )
print('\033[1m  SAC Corrective Env | No Delay | Cutter 4   \033[0m')
print_results(r, p, r_314)

In [ ]:
r,p,r_314 = find_314_non_corr(algo="SAC", env=c6_corrective_pred_no_delay_env, model_dir="models\SAC_corr_no_delay" , )
print('\033[1m  SAC Corrective Env | No Delay | Cutter 6   \033[0m')
print_results(r, p, r_314)

In [ ]:
r,p,r_314 = find_314_non_corr(algo="SAC", env=c4_corrective_pred_with_delay_env, model_dir="models\SAC_corr_with_delay" , )
print('\033[1m  SAC Corrective Env | With Delay | Cutter 4   \033[0m')
print_results(r, p, r_314)

In [ ]:
r,p,r_314 = find_314_non_corr(algo="SAC", env=c6_corrective_pred_with_delay_env, model_dir="models\SAC_corr_with_delay" , )
print('\033[1m  SAC Corrective Env | With Delay | Cutter 6   \033[0m')
print_results(r, p, r_314)

### SAC Non Corr

In [75]:
r,p,r_314 = find_314_non_corr(algo="SAC", env=c4_non_corrective_pred_no_delay_env, model_dir="models\SAC_non_corr_no_delay" , )
print('\033[1m  SAC Non Corrective Env | No Delay | Cutter 4   \033[0m')
print_results(r, p, r_314)

{'cut': 314, 'score': 3112569700.844474}
Reached 314 None
{'cut': 314, 'score': 137990.21655224383}
Reached 314 None
{'cut': 314, 'score': 775.1419368898694}
Reached 314 None
{'cut': 314, 'score': 752.0644359551527}
Reached 314 None
{'cut': 314, 'score': 1413.1886518223914}
Reached 314 None
{'cut': 314, 'score': 4077.608271693491}
Reached 314 None
{'cut': 314, 'score': 5563.735280104939}
Reached 314 None
{'cut': 314, 'score': 2650.167854361031}
Reached 314 None
{'cut': 314, 'score': 18460.197008886204}
Reached 314 None
{'cut': 314, 'score': 56417.46643741999}
Reached 314 None
{'cut': 314, 'score': 930617.4393712846}
Reached 314 None
{'cut': 314, 'score': 10779.825262203014}
Reached 314 None
{'cut': 314, 'score': 8146.882517553841}
Reached 314 None
{'cut': 314, 'score': 37683.41788745915}
Reached 314 None
{'cut': 314, 'score': 10887.223676039039}
Reached 314 None
{'cut': 314, 'score': 18547.217584952097}
Reached 314 None
{'cut': 314, 'score': 157325.7799572666}
Reached 314 None
{'cut': 

In [76]:
r,p,r_314 = find_314_non_corr(algo="SAC", env=c6_non_corrective_pred_no_delay_env, model_dir="models\SAC_non_corr_no_delay" , )
print('\033[1m  SAC Non Corrective Env | No Delay | Cutter 6   \033[0m')
print_results(r, p, r_314)

{'cut': 314, 'score': 32045136.559304975}
Reached 314 None
{'cut': 314, 'score': 1352367.9273216543}
Reached 314 None
{'cut': 314, 'score': 20461500.205586582}
Reached 314 None
{'cut': 314, 'score': 17786245.351090614}
Reached 314 None
{'cut': 314, 'score': 5567386285.209656}
Reached 314 None
{'cut': 314, 'score': 3978999.3507647435}
Reached 314 None
{'cut': 314, 'score': 614069410.4345366}
Reached 314 None
{'cut': 314, 'score': 45746.854076351774}
Reached 314 None
{'cut': 314, 'score': 72722.03189794697}
Reached 314 None
{'cut': 314, 'score': 13494197.928148177}
Reached 314 None
{'cut': 314, 'score': 3140.837366733796}
Reached 314 None
{'cut': 314, 'score': 6254.185614665357}
Reached 314 None
{'cut': 314, 'score': 55968.63652584469}
Reached 314 None
{'cut': 314, 'score': 4083.9950979990153}
Reached 314 None
{'cut': 314, 'score': 9725.406257603883}
Reached 314 None
{'cut': 314, 'score': 1249059.5136913448}
Reached 314 None
{'cut': 314, 'score': 13203.818128797368}
Reached 314 None
{'cu

In [77]:
r,p,r_314 = find_314_non_corr(algo="SAC", env=c4_non_corrective_pred_with_delay_env, model_dir="models\SAC_non_corr_with_delay" , )
print('\033[1m  SAC Non Corrective Env | With Delay | Cutter 4   \033[0m')
print_results(r, p, r_314)

  SAC Non Corrective Env | With Delay | Cutter 4   

Max Reward is :  -23455.729347658093  and it is found at timestep :  20
Min Phm Score is :  11418.283896309002  and it is found at timestep :  20
Max Reward-314 is :  -23455.729347658093  and it is found at timestep :  20


In [78]:
r,p,r_314 = find_314_non_corr(algo="SAC", env=c6_non_corrective_pred_with_delay_env, model_dir="models\SAC_non_corr_with_delay" , )
print('\033[1m  SAC Non Corrective Env | With Delay | Cutter 6   \033[0m')
print_results(r, p, r_314)

  SAC Non Corrective Env | With Delay | Cutter 6   

Max Reward is :  -15048.317533337558  and it is found at timestep :  47
Min Phm Score is :  7712.814000433971  and it is found at timestep :  19
Max Reward-314 is :  -15048.317533337558  and it is found at timestep :  47
